<a href="https://colab.research.google.com/github/Riedel-Gabriela/Tratamento_de_Dados/blob/main/projeto_aquecimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🔥 PROJETO AQUECIMENTO:



#Comentários

*   Ao final de cada reordenação de códigos, substituímos os pd.NA por np.nan pra poder modificar para dict.
*   Este foi um caso visto somente no final e, pra não alterar o codigo todo e correr o risco de deixar passar algum NA despercebido, fizemos a modificação ao final, deixando os códigos com NA e colocando um fillna(np.nan) no fim de cada seção.








#EXECUÇÃO DO AMBIENTE DE TRABALHO

In [ ]:
#Instalando pymongo e pandera
!pip install pymongo[srv]
!pip install pandera

In [ ]:
#Importando as bibliotecas
import pymongo
import pandas as pd
import numpy as np
import pandera as pa

In [ ]:
from pymongo import MongoClient
from datetime import date, timedelta, datetime

In [ ]:
#Conectando com o MongoDB
uri = "mongodb+srv://cluster0.p4kvlxp.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='/content/X509-cert-5675635325549232105.pem')

#CRIANDO OS DATAFRAMES: pandas + gcloud



In [ ]:
#Criando o DataFrame com Pandas
df = pd.read_csv('https://storage.googleapis.com/bc26-gabriela-bucket/projeto_aquecimento/original_1.csv',sep=';')
df2 = pd.read_csv('https://storage.googleapis.com/bc26-gabriela-bucket/projeto_aquecimento/original_2.csv',sep=';')
df3 = pd.read_csv('https://storage.googleapis.com/bc26-gabriela-bucket/projeto_aquecimento/original_3.csv',sep=';')

#MANDANDO OS ARQUIVOS ORIGINAIS: mongoDB

In [ ]:
#Criando as coleções na plataforma mongoDB
'''
db = client['projeto_aquecimento']
colecaooriginal1 = db['original1']
colecaooriginal2 = db['original2']
colecaooriginal3 = db['original3']'''

"\ndb = client['projeto_aquecimento']\ncolecaooriginal1 = db['original1']\ncolecaooriginal2 = db['original2']\ncolecaooriginal3 = db['original3']"

In [ ]:
#Mandar os DataBases originais para o mongoDB

'''
df.reset_index(inplace=True)
df_dict = df.to_dict("records")
colecaooriginal1.insert_many(df_dict)

df2.reset_index(inplace=True)
df2_dict = df2.to_dict("records")
colecaooriginal2.insert_many(df2_dict)

df3.reset_index(inplace=True)
df3_dict = df3.to_dict("records")
colecaooriginal3.insert_many(df3_dict)'''

'\ndf.reset_index(inplace=True)\ndf_dict = df.to_dict("records")\ncolecaooriginal1.insert_many(df_dict)\n\ndf2.reset_index(inplace=True)\ndf2_dict = df2.to_dict("records")\ncolecaooriginal2.insert_many(df2_dict)\n\ndf3.reset_index(inplace=True)\ndf3_dict = df3.to_dict("records")\ncolecaooriginal3.insert_many(df3_dict)'

#🐼 Pandas

#CODANDO OS DATAFRAMES:

In [ ]:
#Ampliando a visão das colunas
pd.set_option('display.max_columns',100)

#TRATAMENTO INICIAL: df

In [ ]:
#Analisando o DataFrame
df

In [ ]:
#Renomeando Colunas
df.dtypes

In [ ]:
df.rename(columns={'CBO':'cbo',
                    'Agente  Causador  Acidente':'agt_causador',
                    'Data Acidente':'dt_acidente',
                    'CID-10':'cid',
                    'CNAE2.0 Empregador':'cnae_cod',
                    'CNAE2.0 Empregador.1':'cnae_desc',
                    'Emitente CAT':'cat',
                    'Espécie do benefício':'beneficio',
                    'Filiação Segurado':'filiacao',
                    'Indica Óbito Acidente':'obito',
                    'Munic Empr':'municipio',
                    'Natureza da Lesão':'lesao',
                    'Origem de Cadastramento CAT':'origem_cad',
                    'Parte Corpo Atingida':'local_lesao',
                    'Sexo':'sexo',
                    'Tipo do Acidente':'tipo_acidente',
                    'UF  Munic.  Acidente':'uf_acidente',
                    'UF Munic. Empregador':'uf_empregador',
                    'Data Acidente.1':'mes_afastamento',
                    'Data Despacho Benefício':'dt_despacho',
                    'Data Acidente.2':'dt_acidente2',
                    'Data Nascimento':'dt_nasc',
                    'Data Emissão CAT':'dt_emissao_cat',
                    'CNPJ/CEI Empregador,,,,,,,,,,,,':'cnpj_cei'
                    }, inplace=True)

In [ ]:
#verificando se ficou tudo ok
df.dtypes

In [ ]:
#Ajustando valores inconsistentes de primeira vista no df
df.replace([',','-'],' ',regex=True, inplace=True)

In [ ]:
#Substituindo valores inconsistentes de primeira vista no df por <NA>
df.replace(['{ñ class}', '{ñ class', '{ñ'],pd.NA,regex=True, inplace=True)

In [ ]:
#Remoção de espaços em branco no df
#Criar uma função que vai remover os espaços em branco no começo e fim dos dados usando dataframe como parâmetro:
def remove_espaco(dataframe):
   
    # olhando os dados das colunas do df
    for i in dataframe.columns:
         
        # checando os dados das colunas
        if dataframe[i].dtype == 'object':
             
            # aplicando função strip nas colunas
            dataframe[i] = dataframe[i].str.strip()
        else:
             
            # se não for do tipo object, passar a diante
            pass
 
# aplicando a função removedora de espaços em branco
remove_espaco(df)

In [ ]:
#Verificando se ficou tudo ok
df.head()

#TRATAMENTO DE COLUNAS: df

##df: Coluna agt_causador

In [ ]:
df['agt_causador'].value_counts()

##df: Coluna dt_acidente

In [ ]:
df['dt_acidente'].value_counts()

##df: Coluna cbo

In [ ]:
df['cbo'].value_counts()

In [ ]:
#separando o código do texto da coluna cbo (Código Brasileiro de Ocupação)
df['cbo_cod'] = df['cbo'].str.slice(0, 7)

In [ ]:
#Renomeando a coluna cbo para cbo_desc ('cbo descrição')
df.rename(columns={'cbo':'cbo_desc'}, inplace=True)

In [ ]:
#removendo os números da coluna cbo_desc
df['cbo_desc'] = df['cbo_desc'].str.replace(r'\d+','', regex=True)

##df: Coluna cbo_cod

In [ ]:
df['cbo_cod'].value_counts()

000000     50940
322205      7125
784205      6490
782510      2513
717020      2266
           ...  
623120         1
123110         1
328105         1
623410         1
213150         1
Name: cbo_cod, Length: 1522, dtype: int64

In [ ]:
#Valores '0000000' foram considerados inconsistencias, mudamos pra NA
df['cbo_cod'].replace(['000000'], pd.NA, regex=True, inplace=True)

##df: Coluna cbo_desc

In [ ]:
df['cbo_desc'].value_counts()

 Não Informado                             50940
 Tec. de Enfermagem                         7125
 Alimentador de Linha de Prod.              6490
 Motorista de Caminhão (Rotas Regionais     2513
 Servente de Obras                          2266
                                           ...  
 Protético Dentário                            1
 Misturador de Chá ou Mate                     1
 Comentarista de Rádio e Televisão             1
 Decorador de Vidro                            1
 Físico (Medicina)                             1
Name: cbo_desc, Length: 1521, dtype: int64

In [ ]:
#Valores 'Não informado' foram considerados inconsistencias, mudamos para NA
df['cbo_desc'].replace(['Não Informado'], pd.NA, regex=True, inplace=True)

##df: Coluna cid

In [ ]:
df['cid'].value_counts()

S62.6 Frat de Outr Dedos                         6376
S61.0 Ferim de Dedos s/Lesao da Unha             5502
S61.1 Ferim de Dedos c/Lesao da Unha             4620
S93.4 Entorse e Distensao do Tornozelo           4059
S52.5 Frat da Extremidade Distal do Radio        2866
                                                 ... 
W00.7 Fazenda                                       1
T82.0 Complic Mecanica Protese Valvular Cardi       1
V46.6 Passageiro Acid Trans                         1
V24.1 Passageiro Acid N Trans                       1
B50   Malaria p/Plasmodium Falciparum               1
Name: cid, Length: 2931, dtype: int64

In [ ]:
#Separando os valores numéricos dos valores escritos da coluna cid
df['cod_cid'] = df['cid'].str.slice(0,6)

In [ ]:
#removendo os números da coluna cid
df['cid'] = df['cid'].str.replace(r'\d+','', regex=True)

In [ ]:
#Renomenaod as colunas para melhor entendimento (cid_desc = cid descrição e cid_cod = cid código)
df.rename(columns={'cid':'cid_desc', 'cod_cid':'cid_cod'}, inplace=True)

##df: Coluna cid_cod

In [ ]:
df['cid_cod'].value_counts()

S62.6     6376
S61.0     5502
S61.1     4620
S93.4     4059
S52.5     2866
          ... 
W00.7        1
T82.0        1
V46.6        1
V24.1        1
B50          1
Name: cid_cod, Length: 2931, dtype: int64

In [ ]:
#Retirando espaços em branco
df['cid_cod'] = df['cid_cod'].str.strip()

##df: Coluna cid_desc

In [ ]:
df['cid_desc'].value_counts()

S. Frat de Outr Dedos                         6376
S. Ferim de Dedos s/Lesao da Unha             5502
S. Ferim de Dedos c/Lesao da Unha             4620
S. Entorse e Distensao do Tornozelo           4059
S. Frat da Extremidade Distal do Radio        2866
                                              ... 
T. Amput Traum Mult Ne                           1
O. Infecc do Saco Amniotico e das Membrana       1
H. Corpo Estranho Ret Intra Ocular Nat Nao       1
N. Glomerulonefrite Prolifer Endocapilar D       1
B   Malaria p/Plasmodium Falciparum              1
Name: cid_desc, Length: 2354, dtype: int64

In [ ]:
#Removendo os valores não numéricos do código CID
df['cid_desc'] = df['cid_desc'].str.replace(r'^..','', regex=True)

In [ ]:
#Verificando se deu certo o código com regex
df['cid_desc'].value_counts()

 Frat de Outr Dedos                     6376
 Ferim de Dedos s/Lesao da Unha         5502
 Ferim de Dedos c/Lesao da Unha         4620
 Entorse e Distensao do Tornozelo       4059
 Frat da Extremidade Distal do Radio    2866
                                        ... 
 Amput Traum do Tronco Nivel Ne            1
 Outr Inseticidas                          1
 Anomalias Geneticas dos Leucocitos        1
 Vacina Bcg                                1
  Malaria p/Plasmodium Falciparum          1
Name: cid_desc, Length: 2331, dtype: int64

In [ ]:
#Retirando os espaços que sobraram
df['cid_desc'] = df['cid_desc'].str.strip()

In [ ]:
#Verificando os parametros da coluna cid_desc, tudo ok
df['cid_desc'].value_counts()

Frat de Outr Dedos                     6376
Ferim de Dedos s/Lesao da Unha         5502
Ferim de Dedos c/Lesao da Unha         4620
Entorse e Distensao do Tornozelo       4059
Frat da Extremidade Distal do Radio    2866
                                       ... 
Transt da Acomodacao                      1
Amput Traum do Tronco Nivel Ne            1
Outr Inseticidas                          1
Anomalias Geneticas dos Leucocitos        1
Malaria p/Plasmodium Falciparum           1
Name: cid_desc, Length: 2330, dtype: int64

##df: Coluna cnae_cod

In [ ]:
df['cnae_cod'].value_counts()

0       53399
8610    11050
4711     3746
4930     2798
8411     2767
        ...  
7416        1
3199        1
2652        1
6130        1
6023        1
Name: cnae_cod, Length: 809, dtype: int64

In [ ]:
#O valor '0' foi considerado inconsistencia, substituímos por NA.
df['cnae_cod'].replace([0],pd.NA,regex=True, inplace=True)

##df: Coluna cnae_desc

In [ ]:
df['cnae_desc'].value_counts()

Atividades de Atendimento Hospitalar            11050
Comercio Varejista de Mercadorias em Geral c     4128
Transporte Rodoviario de Carga                   2798
Administracao Publica em Geral                   2767
Abate de Suinos Aves e Outros Pequenos Anima     2265
                                                ...  
Holdings de Instituicoes Financeiras                1
Transporte Aereo de Carga                           1
Atividades Auxiliares dos Seguros da Previde        1
Extracao de Minerio de Estanho                      1
Comercio Varejista de Discos Cds Dvds e Fit         1
Name: cnae_desc, Length: 606, dtype: int64

##df: Coluna cat

In [ ]:
df['cat'].value_counts()

Empregador             98075
Segurado/Dependente     1384
Sindicato                676
Médico                   252
Autoridade Pública       133
Name: cat, dtype: int64

##df: Coluna beneficio

In [ ]:
df['beneficio'].value_counts()

Pa                                         101201
Auxílio Doenca por Acidente do Trabalho     50845
Auxílio Acidente                               62
Aposent. Invalidez Acidente Trabalho           31
Pensão por Morte Acidente do Trabalho           1
Name: beneficio, dtype: int64

##df: Coluna filiacao

In [ ]:
df['filiacao'].value_counts()

Empregado             100374
Trabalhador Avulso       227
Segurado Especial         41
Name: filiacao, dtype: int64

##df: Coluna obito

In [ ]:
df['obito'].value_counts()

Não    100708
Sim       501
Name: obito, dtype: int64

##df: Coluna municipio

In [ ]:
df['municipio'].value_counts()

000000 Ignorado                 55896
355030 São Paulo                 9051
330455 Rio de Janeiro            3051
310620 Belo Horizonte            2013
410690 Curitiba                  1745
                                ...  
421560 Santa Rosa de Lima Sc        1
230150 Arneiroz                     1
420213 Bela Vista do Toldo          1
220937 Santa Rosa do Piauí          1
431053 Itaara                       1
Name: municipio, Length: 2888, dtype: int64

In [ ]:
#Separando o código do município do nome para melhor avaliação dos dados e criando uma nova coluna cod_uf para os códigos
df['cod_uf'] = df['municipio'].str.slice(0, 6)

In [ ]:
#Apagando o código dos municípios na coluna municipio
df['municipio'] = df['municipio'].str.replace(r'\d+','', regex=True)

In [ ]:
#valor "Ignorado" é uma inconsistencia, será substituído por NA.
df['municipio'].replace(['Ignorado'],pd.NA,regex=True, inplace=True)

##df: Coluna lesao

In [ ]:
df['lesao'].value_counts()

Fratura                                        18798
Corte Laceracao Ferida Contusa Punctura        17682
Lesao Imediata                                 10866
Contusao Esmagamento (Superficie Cutanea I     10231
Escoriacao Abrasao (Ferimento Superficial)      9005
Distensao Torcao                                7321
Luxacao                                         5876
Outras Lesoes Nic                               5659
Doenca Contagiosa ou Infecciosa (Tuberculos     3160
Lesao Imediata Nic                              2720
Queimadura ou Escaldadura   Efeito de Tempe     2470
Lesoes Multiplas                                1811
Doenca Nic                                      1792
Inflamacao de Articulacao Tendao ou Muscul       843
Amputacao ou Enucleacao                          781
Queimadura Quimica (Lesao de Tecido Provoca      666
Concussao Cerebral                               340
Choque Eletrico e Eletroplessao (Eletrocuss      284
Perda ou Diminuicao de Sentido (Audicao Vi    

##df: Coluna origem_cad

In [ ]:
df['origem_cad'].value_counts()

Internet    101209
Name: origem_cad, dtype: int64

In [ ]:
#Coluna com valor constante. Optamos por drop
df.drop(['origem_cad'], axis=1, inplace=True)

##df: Coluna local_lesao

In [ ]:
df['local_lesao'].value_counts()

Dedo                                             23346
Pe (Exceto Artelhos)                              8341
Mao (Exceto Punho ou Dedos)                       7154
Joelho                                            5076
Partes Multiplas   Aplica Se Quando Mais de U     4422
Perna (Entre O Tornozelo e a Pelvis)              4150
Braco (Entre O Punho a O Ombro)                   4020
Articulacao do Tornozelo                          3638
Perna (Do Tornozelo Exclusive ao Joelho Ex        3343
Olho (Inclusive Nervo Otico e Visao)              3148
Aparelho Respiratorio                             3016
Ombro                                             2981
Punho                                             2657
Cabeca Nic                                        2422
Antebraco (Entre O Punho e O Cotovelo)            2206
Dorso (Inclusive Musculos Dorsais Coluna e M      2185
Membros Superiores Partes Multiplas (Qualque      2069
Face Partes Multiplas (Qualquer Combinacao D      1534
Cabeca Par

##df: Coluna sexo

In [ ]:
df['sexo'].value_counts()

Masculino        100405
Feminino          51281
Não Informado       452
Indeterminado         2
Name: sexo, dtype: int64

In [ ]:
#Valores não informados consideramos como inconsistencias e mudamos por NA
#Valores indeterminados permanecem indeterminados
df['sexo'].replace(['Não Informado'], pd.NA, regex=True, inplace=True)

##df: Coluna tipo_acidente

In [ ]:
df['tipo_acidente'].value_counts()

Típico      74369
Ignorado    50768
Trajeto     22508
Doença       4495
Name: tipo_acidente, dtype: int64

##df: Coluna uf_acidente

In [ ]:
df['uf_acidente'].value_counts()

Maranhão            51419
Rondônia            16866
Roraima             10102
Tocantins            9847
Zerado               4966
Pará                 3365
Acre                 2054
Ceará                1925
Amazonas             1115
Pernambuco            609
Amapá                 508
Sergipe               457
Piauí                 346
Alagoas               259
Rio Grande Norte      233
Paraíba               178
Name: uf_acidente, dtype: int64

In [ ]:
#Entendemos Zerado como uma inconsistencia, mudamos pra NA
df['uf_acidente'].replace(['Zerado'], pd.NA, regex=True, inplace=True)

##df: Coluna uf_empregador

In [ ]:
df['uf_empregador'].value_counts()

Zerado                 55897
São Paulo              34865
Minas Gerais           10172
Rio Grande do Sul       8018
Paraná                  7891
Santa Catarina          7440
Rio de Janeiro          6047
Goiás                   2648
Bahia                   2432
Espírito Santo          2150
Pernambuco              1891
Ceará                   1859
Mato Grosso             1753
Mato Grosso do Sul      1510
Pará                    1367
Distrito Federal        1181
Amazonas                 927
Maranhão                 628
Rio Grande do Norte      620
Alagoas                  484
Paraíba                  482
Rondônia                 472
Tocantins                432
Piauí                    305
Sergipe                  300
Amapá                    150
Roraima                  122
Acre                      97
Name: uf_empregador, dtype: int64

In [ ]:
#Valor Zerado foi considerado inconsistencia e mudamos para NA
df['uf_empregador'].replace(['Zerado'], pd.NA, regex=True, inplace=True)

##df: Coluna mes_afastamento

In [ ]:
df['mes_afastamento'].value_counts()

2022/03    51210
2022/06    39459
2022/02    22961
2022/01    17891
2022/05    12828
2022/04     7791
Name: mes_afastamento, dtype: int64

##df: Coluna dt_despacho

In [ ]:
df['dt_despacho'].value_counts()

0000/00    101201
2022/03     15505
2022/06     12439
2022/05      9030
2022/02      6609
2022/04      5379
2022/07      1201
2022/01       776
Name: dt_despacho, dtype: int64

In [ ]:
#Valor 0000/00 foi considerado inconsistencia, mudamos pra NA
df['dt_despacho'].replace(['0000/00'], pd.NA, regex=True, inplace=True)

##df: Coluna dt_acidente2

In [ ]:
df['dt_acidente2'].value_counts()

10/03/2022    2402
08/03/2022    2400
09/03/2022    2351
07/03/2022    2350
03/03/2022    2314
              ... 
03/04/2022      84
15/05/2022      82
01/05/2022      81
08/05/2022      81
17/04/2022      67
Name: dt_acidente2, Length: 181, dtype: int64

##df: Coluna dt_nasc

In [ ]:
df['dt_nasc'].value_counts()

00/00/0000    194
07/09/1981     30
15/01/1982     29
27/04/1998     28
13/03/1997     28
             ... 
14/03/1950      1
02/12/1951      1
24/01/1955      1
23/12/1966      1
16/02/1963      1
Name: dt_nasc, Length: 17628, dtype: int64

In [ ]:
#Valor 00/00/0000 foi considerado inconsistencia, mudamos pra NA
df['dt_nasc'].replace(['00/00/0000'], pd.NA, regex=True, inplace=True)

##df: Coluna dt_emissao_cat

In [ ]:
df['dt_emissao_cat'].value_counts()

00/00/0000    50931
28/06/2022     2616
22/03/2022     2531
27/06/2022     2515
21/06/2022     2510
              ...  
26/06/2022       42
20/03/2022       40
06/03/2022       35
27/03/2022       33
19/06/2022       32
Name: dt_emissao_cat, Length: 62, dtype: int64

In [ ]:
#Valor 00/00/0000 foi considerado inconsistencia, mudamos pra NA
df['dt_emissao_cat'].replace(['00/00/0000'], pd.NA, regex=True, inplace=True)

##df: Coluna cnpj_cei

In [ ]:
df['cnpj_cei'].value_counts()

00.000.000.000.000    128827
57.571.275.002.570       377
47.673.793.010.217       102
28.549.483.000.105        99
60.701.190.000.104        99
                       ...  
27.957.196.000.163         1
76.354.281.001.467         1
00.055.814.573.104         1
07.313.870.000.160         1
00.030.808.740.210         1
Name: cnpj_cei, Length: 14662, dtype: int64

In [ ]:
#Valor 00.000.000.000.000 foi considerado inconsistencia, mudamos pra NA
df['cnpj_cei'].replace(['00.000.000.000.000'], pd.NA, regex=True, inplace=True)

##df: Coluna cod_uf

In [ ]:
df['cod_uf'].value_counts()

000000    55896
355030     9051
330455     3051
310620     2013
410690     1745
          ...  
421560        1
230150        1
420213        1
220937        1
431053        1
Name: cod_uf, Length: 2888, dtype: int64

In [ ]:
#Valor 000000 foi considerado inconsistencia, mudamos pra NA
df['cod_uf'].replace(['000000'], pd.NA, regex=True, inplace=True)

#REORDENANDO: df

In [ ]:
df = df[['dt_acidente','dt_acidente2','dt_despacho','mes_afastamento','dt_emissao_cat','sexo','dt_nasc','uf_empregador','uf_acidente','cod_uf','municipio','filiacao','beneficio','cat','tipo_acidente','agt_causador','lesao','local_lesao','obito','cid_cod','cid_desc','cbo_cod','cbo_desc','cnae_cod','cnae_desc','cnpj_cei']]

In [ ]:
df.head(1)

,dt_acidente,dt_acidente2,dt_despacho,mes_afastamento,dt_emissao_cat,sexo,dt_nasc,uf_empregador,uf_acidente,cod_uf,municipio,filiacao,beneficio,cat,tipo_acidente,agt_causador,lesao,local_lesao,obito,cid_cod,cid_desc,cbo_cod,cbo_desc,cnae_cod,cnae_desc,cnpj_cei
0,2022/01,20/01/2022,<NA>,2022/01,01/03/2022,Feminino,02/08/1970,São Paulo,Maranhão,354780,Santo André Sp,Empregado,Pa,Empregador,Doença,<NA>,Doenca Contagiosa ou Infecciosa (Tuberculos,Aparelho Respiratorio,Não,B34.2,Infecc p/Coronavirus Ne,515105,Agente Comunitário de Saúde,8630,Atividades de Atencao Ambulatorial Executadas,57.571.275.002.570


In [ ]:
#Percebemos algumas datas redundantes, como por exemplo a coluna de dt_acidente e dt_acidente2, decidimos dropar a menos específica.
df.drop(['dt_acidente'], axis=1, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
#renomeamos a coluna dt_acidente2 para dt_acidente
df.rename(columns={'dt_acidente2':'dt_acidente'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
df = df.fillna(np.nan)

In [ ]:
#ultima remoção de possíveis espaços em branco
remove_espaco(df)

In [ ]:
df.head(2)

,dt_acidente,dt_despacho,mes_afastamento,dt_emissao_cat,sexo,dt_nasc,uf_empregador,uf_acidente,cod_uf,municipio,filiacao,beneficio,cat,tipo_acidente,agt_causador,lesao,local_lesao,obito,cid_cod,cid_desc,cbo_cod,cbo_desc,cnae_cod,cnae_desc,cnpj_cei
0,20/01/2022,NaN,2022/01,01/03/2022,Feminino,02/08/1970,São Paulo,Maranhão,354780,Santo André Sp,Empregado,Pa,Empregador,Doença,NaN,Doenca Contagiosa ou Infecciosa (Tuberculos,Aparelho Respiratorio,Não,B34.2,Infecc p/Coronavirus Ne,515105,Agente Comunitário de Saúde,8630.0,Atividades de Atencao Ambulatorial Executadas,57.571.275.002.570
1,25/02/2022,NaN,2022/02,01/03/2022,Masculino,16/12/1990,São Paulo,Maranhão,355030,São Paulo,Empregado,Pa,Segurado/Dependente,Trajeto,Motocicleta Motoneta,Fratura,Braco (Entre O Punho a O Ombro),Não,S52.5,Frat da Extremidade Distal do Radio,519110,Motociclista no Transporte de Document,4930.0,Transporte Rodoviario de Carga,19.578.086.000.162


In [ ]:
#Drop de linhas duplicadas
df.drop_duplicates(inplace=True)

#TRATAMENTO INICIAL: df2

In [ ]:
#Analisando o DataFrame
df2

,Agente Causador Acidente,Data Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,Munic Empr,Natureza da Lesão,Origem de Cadastramento CAT,Parte Corpo Atingida,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,"CNPJ/CEI Empregador,,,,,,,,,,,,"
0,{ñ class},2022/05,322205-Tec. de Enfermagem,Y28.2 Escolas Outr Instit Areas Admin Publica,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,Lesao Imediata,Internet,Dedo,Masculino,Típico,Maranhão,São Paulo,2022/05,0000/00,26/05/2022,13/11/1989,01/06/2022,"57.571.275.002.570,,,,,,,,,,,,"
1,Rua e Estrada - Superficie Utilizada para Sus,2022/05,992225-Auxiliar Geral de Conservação de Vias,S93.4 Entorse e Distensao do Tornozelo,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,Luxacao,Internet,"Perna (Do Tornozelo,Exclusive,ao Joelho,Ex",Feminino,Trajeto,Maranhão,São Paulo,2022/05,0000/00,26/05/2022,19/04/1990,01/06/2022,"57.571.275.002.570,,,,,,,,,"
2,{ñ class},2022/05,{ñ class},B34.2 Infecc p/Coronavirus Ne,8513,Ensino Fundamental,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,Doenca Contagiosa ou Infecciosa (Tuberculos,Internet,Aparelho Respiratorio,Feminino,Doença,Maranhão,São Paulo,2022/05,0000/00,29/05/2022,13/09/1960,01/06/2022,"47.715.347.000.184,,,,,,,,,,,,"
3,"Revestimento Ceramico (Azulejo,Mosaico,Etc.",2022/05,783225-Ajudante de Motorista,S90.3 Contusao de Outr Partes e Partes Ne Do,4744,"Comercio Varejista de Ferragens,Madeira e Ma",Empregador,Pa,Empregado,Não,350210-Andradina,"Contusao,Esmagamento (Superficie Cutanea I",Internet,Pe (Exceto Artelhos),Masculino,Típico,Maranhão,São Paulo,2022/05,0000/00,31/05/2022,17/07/1994,01/06/2022,"38.300.481.000.123,,,,,,,,"
4,Calcada ou Caminho para Pedestre - Superficie,2022/05,322205-Tec. de Enfermagem,S51.0 Ferim do Cotovelo,8610,Atividades de Atendimento Hospitalar,Empregador,Pa,Empregado,Não,350950-Campinas,"Escoriacao,Abrasao (Ferimento Superficial)",Internet,Cotovelo,Feminino,Trajeto,Maranhão,São Paulo,2022/05,0000/00,30/05/2022,30/01/1992,01/06/2022,"46.020.301.000.269,,,,,,,,,,,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89597,{ñ class},2022/04,000000-Não Informado,S82.6 Frat do Maleolo Lateral,0,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class,{ñ class},Masculino,Ignorado,Sergipe,Zerado,2022/04,2022/06,12/04/2022,08/11/1965,00/00/0000,"00.000.000.000.000,,,,,,,,,,,,"
89598,{ñ class},2022/04,000000-Não Informado,S82.1 Frat da Extremidade Proximal da Tibia,0,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class,{ñ class},Masculino,Ignorado,Sergipe,Zerado,2022/04,2022/06,30/04/2022,21/11/1985,00/00/0000,"00.000.000.000.000,,,,,,,,,,,,"
89599,{ñ class},2022/05,000000-Não Informado,S62.6 Frat de Outr Dedos,0,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class,{ñ class},Masculino,Ignorado,Sergipe,Zerado,2022/05,2022/06,05/05/2022,27/01/1992,00/00/0000,"00.000.000.000.000,,,,,,,,,,,,"
89600,{ñ class},2022/05,000000-Não Informado,S82.6 Frat do Maleolo Lateral,0,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class,{ñ class},Masculino,Ignorado,Sergipe,Zerado,2022/05,2022/06,15/05/2022,14/01/1975,00/00/0000,"00.000.000.000.000,,,,,,,,,,,,"


In [ ]:
#Renomeando Colunas
df2.dtypes

Agente  Causador  Acidente         object
Data Acidente                      object
CBO                                object
CID-10                             object
CNAE2.0 Empregador                  int64
CNAE2.0 Empregador.1               object
Emitente CAT                       object
Espécie do benefício               object
Filiação Segurado                  object
Indica Óbito Acidente              object
Munic Empr                         object
Natureza da Lesão                  object
Origem de Cadastramento CAT        object
Parte Corpo Atingida               object
Sexo                               object
Tipo do Acidente                   object
UF  Munic.  Acidente               object
UF Munic. Empregador               object
Data Acidente.1                    object
Data Despacho Benefício            object
Data Acidente.2                    object
Data Nascimento                    object
Data Emissão CAT                   object
CNPJ/CEI Empregador,,,,,,,,,,,,   

In [ ]:
df2.rename(columns={'CBO':'cbo',
                    'Agente  Causador  Acidente':'agt_causador',
                    'Data Acidente':'dt_acidente',
                    'CID-10':'cid',
                    'CNAE2.0 Empregador':'cnae_cod',
                    'CNAE2.0 Empregador.1':'cnae_desc',
                    'Emitente CAT':'cat',
                    'Espécie do benefício':'beneficio',
                    'Filiação Segurado':'filiacao',
                    'Indica Óbito Acidente':'obito',
                    'Munic Empr':'municipio',
                    'Natureza da Lesão':'lesao',
                    'Origem de Cadastramento CAT':'origem_cad',
                    'Parte Corpo Atingida':'local_lesao',
                    'Sexo':'sexo',
                    'Tipo do Acidente':'tipo_acidente',
                    'UF  Munic.  Acidente':'uf_acidente',
                    'UF Munic. Empregador':'uf_empregador',
                    'Data Acidente.1':'mes_afastamento',
                    'Data Despacho Benefício':'dt_despacho',
                    'Data Acidente.2':'dt_acidente2',
                    'Data Nascimento':'dt_nasc',
                    'Data Emissão CAT':'dt_emissao_cat',
                    'CNPJ/CEI Empregador,,,,,,,,,,,,':'cnpj_cei'
                    }, inplace=True)

In [ ]:
#verificando se ficou tudo ok
df2.dtypes

agt_causador       object
dt_acidente        object
cbo                object
cid                object
cnae_cod            int64
cnae_desc          object
cat                object
beneficio          object
filiacao           object
obito              object
municipio          object
lesao              object
origem_cad         object
local_lesao        object
sexo               object
tipo_acidente      object
uf_acidente        object
uf_empregador      object
mes_afastamento    object
dt_despacho        object
dt_acidente2       object
dt_nasc            object
dt_emissao_cat     object
cnpj_cei           object
dtype: object

In [ ]:
#Ajustando valores inconsistentes de primeira vista no df
df2.replace([',','-'],' ',regex=True, inplace=True)

In [ ]:
#Substituindo valores inconsistentes de primeira vista no df por <NA>
df2.replace(['{ñ class}', '{ñ class', '{ñ'],pd.NA,regex=True, inplace=True)

In [ ]:
#Remoção de espaços em branco no df
# aplicando a função removedora de espaços em branco
remove_espaco(df2)

In [ ]:
#Verificando se ficou tudo ok
df2.head()

,agt_causador,dt_acidente,cbo,cid,cnae_cod,cnae_desc,cat,beneficio,filiacao,obito,municipio,lesao,origem_cad,local_lesao,sexo,tipo_acidente,uf_acidente,uf_empregador,mes_afastamento,dt_despacho,dt_acidente2,dt_nasc,dt_emissao_cat,cnpj_cei
0,<NA>,2022/05,322205 Tec. de Enfermagem,Y28.2 Escolas Outr Instit Areas Admin Publica,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780 Santo André Sp,Lesao Imediata,Internet,Dedo,Masculino,Típico,Maranhão,São Paulo,2022/05,0000/00,26/05/2022,13/11/1989,01/06/2022,57.571.275.002.570
1,Rua e Estrada Superficie Utilizada para Sus,2022/05,992225 Auxiliar Geral de Conservação de Vias,S93.4 Entorse e Distensao do Tornozelo,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780 Santo André Sp,Luxacao,Internet,Perna (Do Tornozelo Exclusive ao Joelho Ex,Feminino,Trajeto,Maranhão,São Paulo,2022/05,0000/00,26/05/2022,19/04/1990,01/06/2022,57.571.275.002.570
2,<NA>,2022/05,<NA>,B34.2 Infecc p/Coronavirus Ne,8513,Ensino Fundamental,Empregador,Pa,Empregado,Não,354780 Santo André Sp,Doenca Contagiosa ou Infecciosa (Tuberculos,Internet,Aparelho Respiratorio,Feminino,Doença,Maranhão,São Paulo,2022/05,0000/00,29/05/2022,13/09/1960,01/06/2022,47.715.347.000.184
3,Revestimento Ceramico (Azulejo Mosaico Etc.,2022/05,783225 Ajudante de Motorista,S90.3 Contusao de Outr Partes e Partes Ne Do,4744,Comercio Varejista de Ferragens Madeira e Ma,Empregador,Pa,Empregado,Não,350210 Andradina,Contusao Esmagamento (Superficie Cutanea I,Internet,Pe (Exceto Artelhos),Masculino,Típico,Maranhão,São Paulo,2022/05,0000/00,31/05/2022,17/07/1994,01/06/2022,38.300.481.000.123
4,Calcada ou Caminho para Pedestre Superficie,2022/05,322205 Tec. de Enfermagem,S51.0 Ferim do Cotovelo,8610,Atividades de Atendimento Hospitalar,Empregador,Pa,Empregado,Não,350950 Campinas,Escoriacao Abrasao (Ferimento Superficial),Internet,Cotovelo,Feminino,Trajeto,Maranhão,São Paulo,2022/05,0000/00,30/05/2022,30/01/1992,01/06/2022,46.020.301.000.269


#TRATAMENTO DE COLUNAS: df2

##df2: Coluna agt_causador

In [ ]:
df2['agt_causador'].value_counts()

Motocicleta Motoneta                             2521
Impacto de Pes. Contra Objeto em Movimento       2325
Rua e Estrada   Superficie Utilizada para Sus    1902
Impacto de Pes. Contra Objeto Parado             1770
Chao   Superficie Utilizada para Sustentar Pe    1596
                                                 ... 
Turbina                                             1
Legume Verdura e Derivados                          1
Ferro de Passar  Ferramenta Portatil com Forc       1
Nafta e Solvente de Nafta (Eter de Petroleo         1
Equip. Magnetico   Equip. Eletrico                  1
Name: agt_causador, Length: 283, dtype: int64

##df2: Coluna dt_acidente

In [ ]:
df2['dt_acidente'].value_counts()

2022/06    39459
2022/05    12828
2022/03    10221
2022/02     9656
2022/01     9647
2022/04     7791
Name: dt_acidente, dtype: int64

##df2: Coluna cbo

In [ ]:
df2['cbo'].value_counts()

000000 Não Informado                             39919
322205 Tec. de Enfermagem                         3642
784205 Alimentador de Linha de Prod.              3066
782510 Motorista de Caminhão (Rotas Regionais     1238
717020 Servente de Obras                          1140
                                                 ...  
322305 Tec. em óptica e Optometria                   1
771115 Maquetista na Marcenaria                      1
234730 Prof. Direito do Ensino Superior              1
214505 Eng. Químico                                  1
411035 Auxiliar Estatística                          1
Name: cbo, Length: 1345, dtype: int64

In [ ]:
#separando o código do texto da coluna cbo (Código Brasileiro de Ocupação)
df2['cbo_cod'] = df2['cbo'].str.slice(0, 7)

In [ ]:
#Renomeando a coluna cbo para cbo_desc ('cbo descrição')
df2.rename(columns={'cbo':'cbo_desc'}, inplace=True)

In [ ]:
#removendo os números da coluna cbo_desc
df2['cbo_desc'] = df2['cbo_desc'].str.replace(r'\d+','', regex=True)

##df2: Coluna cbo_cod

In [ ]:
df2['cbo_cod'].value_counts()

000000     39919
322205      3642
784205      3066
782510      1238
717020      1140
           ...  
322305         1
771115         1
234730         1
214505         1
411035         1
Name: cbo_cod, Length: 1345, dtype: int64

In [ ]:
#Valores '0000000' foram considerados inconsistencias, mudamos pra NA
df2['cbo_cod'].replace(['000000'], pd.NA, regex=True, inplace=True)

##df2: Coluna cbo_desc

In [ ]:
df2['cbo_desc'].value_counts()

 Não Informado                             39919
 Tec. de Enfermagem                         3642
 Alimentador de Linha de Prod.              3066
 Motorista de Caminhão (Rotas Regionais     1238
 Servente de Obras                          1140
                                           ...  
 Oper. Máquina de Extração Contínua (Mi        1
 Piloto Fluvial                                1
 Tec. em óptica e Optometria                   1
 Maquetista na Marcenaria                      1
 Auxiliar Estatística                          1
Name: cbo_desc, Length: 1344, dtype: int64

In [ ]:
#Valores 'Não informado' foram considerados inconsistencias, mudamos para NA
df2['cbo_desc'].replace(['Não Informado'], pd.NA, regex=True, inplace=True)

##df2: Coluna cid

In [ ]:
df2['cid'].value_counts()

S62.6 Frat de Outr Dedos                         4090
S61.0 Ferim de Dedos s/Lesao da Unha             2729
S61.1 Ferim de Dedos c/Lesao da Unha             2306
S93.4 Entorse e Distensao do Tornozelo           2226
S52.5 Frat da Extremidade Distal do Radio        1889
                                                 ... 
Y87.1 Sequelas de Uma Agressao                      1
L98.0 Granuloma Piogenico                           1
T26.8 Corrosao de Outr Partes do Olho e Anexo       1
M90.7 Frat Ossea em Doenc Neoplasicas               1
B50   Malaria p/Plasmodium Falciparum               1
Name: cid, Length: 2436, dtype: int64

In [ ]:
#Separando os valores numéricos dos valores escritos da coluna cid
df2['cod_cid'] = df2['cid'].str.slice(0,6)

In [ ]:
#removendo os números da coluna cid
df2['cid'] = df2['cid'].str.replace(r'\d+','', regex=True)

In [ ]:
#Renomenaod as colunas para melhor entendimento (cid_desc = cid descrição e cid_cod = cid código)
df2.rename(columns={'cid':'cid_desc', 'cod_cid':'cid_cod'}, inplace=True)

##df2: Coluna cid_cod

In [ ]:
df2['cid_cod'].value_counts()

S62.6     4090
S61.0     2729
S61.1     2306
S93.4     2226
S52.5     1889
          ... 
Y87.1        1
L98.0        1
T26.8        1
M90.7        1
B50          1
Name: cid_cod, Length: 2436, dtype: int64

In [ ]:
#Retirando espaços em branco
df2['cid_cod'] = df2['cid_cod'].str.strip()

##df2: Coluna cid_desc

In [ ]:
df2['cid_desc'].value_counts()

S. Frat de Outr Dedos                         4090
S. Ferim de Dedos s/Lesao da Unha             2729
S. Ferim de Dedos c/Lesao da Unha             2306
S. Entorse e Distensao do Tornozelo           2226
S. Frat da Extremidade Distal do Radio        1889
                                              ... 
K. Hernia Abdominal Ne c/Obstrucao s/Gangr       1
I. Sindr de Pre Excitacao                        1
A. Infecc p/Clamid Pelviper e Outr Org Gen       1
Y. Sequelas de Uma Agressao                      1
B   Malaria p/Plasmodium Falciparum              1
Name: cid_desc, Length: 2031, dtype: int64

In [ ]:
#Removendo os valores não numéricos do código CID
df2['cid_desc'] = df2['cid_desc'].str.replace(r'^..','', regex=True)

In [ ]:
#Verificando se deu certo o código com regex
df2['cid_desc'].value_counts()

 Frat de Outr Dedos                         4090
 Ferim de Dedos s/Lesao da Unha             2729
 Ferim de Dedos c/Lesao da Unha             2306
 Entorse e Distensao do Tornozelo           2226
 Frat da Extremidade Distal do Radio        1889
                                            ... 
 Tuberc Miliar Aguda de Localiz Unica Es       1
 Anestesicos Intravenosos                      1
 Insuf Renal Aguda Ne                          1
 Exame Seguim Apos Cirurgia p/Outr Afecc       1
  Malaria p/Plasmodium Falciparum              1
Name: cid_desc, Length: 2010, dtype: int64

In [ ]:
#Retirando os espaços que sobraram
df2['cid_desc'] = df2['cid_desc'].str.strip()

In [ ]:
#Verificando os parametros da coluna cid_desc, tudo ok
df2['cid_desc'].value_counts()

Frat de Outr Dedos                         4090
Ferim de Dedos s/Lesao da Unha             2729
Ferim de Dedos c/Lesao da Unha             2306
Entorse e Distensao do Tornozelo           2226
Frat da Extremidade Distal do Radio        1889
                                           ... 
Sequelas de Ferim da Cabeca                   1
Compr Signif Comport Req Vigilancia Ou        1
Projetil de Revolver                          1
Traum dos Nervos Perifericos do Pescoco       1
Malaria p/Plasmodium Falciparum               1
Name: cid_desc, Length: 2009, dtype: int64

##df2: Coluna cnae_cod

In [ ]:
df2['cnae_cod'].value_counts()

0       41096
8610     5547
4711     1891
8411     1447
4930     1360
        ...  
5519        1
853         1
1939        1
524         1
6023        1
Name: cnae_cod, Length: 734, dtype: int64

In [ ]:
#O valor '0' foi considerado inconsistencia, substituímos por NA.
df2['cnae_cod'].replace([0],pd.NA,regex=True, inplace=True)

##df2: Coluna cnae_desc

In [ ]:
df2['cnae_desc'].value_counts()

Atividades de Atendimento Hospitalar             5547
Comercio Varejista de Mercadorias em Geral c     2081
Administracao Publica em Geral                   1447
Transporte Rodoviario de Carga                   1360
Abate de Suinos Aves e Outros Pequenos Anima     1072
                                                 ... 
Construcao de Redes de Transportes por Dutos        1
Edicao Integrada a Impressao de Revistas            1
Transporte por Navegacao Interior de Passagei       1
Servicos de Reservas e Outros Servicos de Tur       1
Atividades de Administracao de Fundos por Con       1
Name: cnae_desc, Length: 593, dtype: int64

##df2: Coluna cat

In [ ]:
df2['cat'].value_counts()

Empregador             48054
Segurado/Dependente      755
Sindicato                328
Médico                   142
Autoridade Pública        82
Name: cat, dtype: int64

##df2: Coluna beneficio

In [ ]:
df2['beneficio'].value_counts()

Pa                                         49689
Auxílio Doenca por Acidente do Trabalho    39838
Auxílio Acidente                              50
Aposent. Invalidez Acidente Trabalho          25
Name: beneficio, dtype: int64

##df2: Coluna filiacao

In [ ]:
df2['filiacao'].value_counts()

Empregado             49287
Trabalhador Avulso      104
Segurado Especial        20
Name: filiacao, dtype: int64

##df2: Coluna obito

In [ ]:
df2['obito'].value_counts()

Não    49419
Sim      270
Name: obito, dtype: int64

##df2: Coluna municipio

In [ ]:
df2['municipio'].value_counts()

000000 Ignorado                    41986
355030 São Paulo                    4479
330455 Rio de Janeiro               1450
310620 Belo Horizonte                984
431490 Porto Alegre                  895
                                   ...  
411722 Nova Santa Rosa                 1
261260 Santa Maria da Boa Vista        1
110094 Cujubim                         1
410090 Amaporã                         1
431053 Itaara                          1
Name: municipio, Length: 2380, dtype: int64

In [ ]:
#Separando o código do município do nome para melhor avaliação dos dados e criando uma nova coluna cod_uf para os códigos
df2['cod_uf'] = df2['municipio'].str.slice(0, 6)

In [ ]:
#Apagando o código dos municípios na coluna municipio
df2['municipio'] = df2['municipio'].str.replace(r'\d+','', regex=True)

In [ ]:
#valor "Ignorado" é uma inconsistencia, será substituído por NA.
df2['municipio'].replace(['Ignorado'],pd.NA,regex=True, inplace=True)

##df2: Coluna lesao

In [ ]:
df2['lesao'].value_counts()

Fratura                                        9367
Corte Laceracao Ferida Contusa Punctura        8365
Lesao Imediata                                 5258
Contusao Esmagamento (Superficie Cutanea I     5131
Escoriacao Abrasao (Ferimento Superficial)     4205
Distensao Torcao                               3534
Outras Lesoes Nic                              2803
Luxacao                                        2798
Doenca Contagiosa ou Infecciosa (Tuberculos    2117
Lesao Imediata Nic                             1307
Queimadura ou Escaldadura   Efeito de Tempe    1138
Doenca Nic                                      895
Lesoes Multiplas                                889
Inflamacao de Articulacao Tendao ou Muscul      462
Amputacao ou Enucleacao                         379
Queimadura Quimica (Lesao de Tecido Provoca     294
Concussao Cerebral                              173
Perda ou Diminuicao de Sentido (Audicao Vi      168
Choque Eletrico e Eletroplessao (Eletrocuss     117
Envenenament

##df2: Coluna origem_cad

In [ ]:
df2['origem_cad'].value_counts()

Internet    49689
Name: origem_cad, dtype: int64

In [ ]:
#Coluna com valor constante. Optamos por drop
df2.drop(['origem_cad'], axis=1, inplace=True)

##df2: Coluna local_lesao

In [ ]:
df2['local_lesao'].value_counts()

Dedo                                             11399
Pe (Exceto Artelhos)                              3884
Mao (Exceto Punho ou Dedos)                       3520
Joelho                                            2561
Partes Multiplas   Aplica Se Quando Mais de U     2161
Perna (Entre O Tornozelo e a Pelvis)              2079
Aparelho Respiratorio                             2000
Braco (Entre O Punho a O Ombro)                   1881
Articulacao do Tornozelo                          1726
Perna (Do Tornozelo Exclusive ao Joelho Ex        1609
Ombro                                             1549
Olho (Inclusive Nervo Otico e Visao)              1414
Punho                                             1252
Cabeca Nic                                        1183
Dorso (Inclusive Musculos Dorsais Coluna e M      1079
Antebraco (Entre O Punho e O Cotovelo)             983
Membros Superiores Partes Multiplas (Qualque       948
Face Partes Multiplas (Qualquer Combinacao D       786
Cabeca Par

##df2: Coluna sexo

In [ ]:
df2['sexo'].value_counts()

Masculino        58961
Feminino         30323
Não Informado      317
Indeterminado        1
Name: sexo, dtype: int64

In [ ]:
#Valores não informados consideramos como inconsistencias e mudamos por NA
#Valores indeterminados permanecem indeterminados
df2['sexo'].replace(['Não Informado'], pd.NA, regex=True, inplace=True)

##df2: Coluna tipo_acidente

In [ ]:
df2['tipo_acidente'].value_counts()

Ignorado    39788
Típico      35724
Trajeto     11257
Doença       2833
Name: tipo_acidente, dtype: int64

##df2: Coluna uf_acidente

In [ ]:
df2['uf_acidente'].value_counts()

Maranhão            30217
Rondônia            10290
Tocantins            5852
Roraima              5830
Zerado               2073
Pará                 2008
Ceará                1230
Acre                 1181
Amazonas              693
Pernambuco            363
Amapá                 343
Sergipe               237
Piauí                 194
Rio Grande Norte      158
Alagoas               142
Paraíba                60
Name: uf_acidente, dtype: int64

In [ ]:
#Entendemos Zerado como uma inconsistencia, mudamos pra NA
df2['uf_acidente'].replace(['Zerado'], pd.NA, regex=True, inplace=True)

##df2: Coluna uf_empregador

In [ ]:
df2['uf_empregador'].value_counts()

Zerado                 41986
São Paulo              17413
Minas Gerais            4985
Rio Grande do Sul       3966
Paraná                  3858
Santa Catarina          3555
Rio de Janeiro          3033
Goiás                   1293
Bahia                   1163
Espírito Santo          1064
Ceará                   1001
Mato Grosso              895
Pernambuco               840
Mato Grosso do Sul       738
Pará                     654
Distrito Federal         618
Amazonas                 483
Rio Grande do Norte      336
Maranhão                 331
Rondônia                 254
Paraíba                  235
Alagoas                  228
Tocantins                214
Piauí                    171
Sergipe                  150
Acre                      52
Roraima                   48
Amapá                     38
Name: uf_empregador, dtype: int64

In [ ]:
#Entendemos Zerado como uma inconsistencia, mudamos pra NA
df2['uf_empregador'].replace(['Zerado'], pd.NA, regex=True, inplace=True)

##df2: Coluna mes_afastamento

In [ ]:
df2['mes_afastamento'].value_counts()

2022/06    39459
2022/05    12828
2022/03    10221
2022/02     9656
2022/01     9647
2022/04     7791
Name: mes_afastamento, dtype: int64

##df2: Coluna dt_despacho

In [ ]:
df2['dt_despacho'].value_counts()

0000/00    49689
2022/06    12439
2022/05     9030
2022/03     8296
2022/04     4970
2022/02     3577
2022/07     1200
2022/01      401
Name: dt_despacho, dtype: int64

In [ ]:
#Valor 0000/00 foi considerado inconsistencia, mudamos pra NA
df2['dt_despacho'].replace(['0000/00'], pd.NA, regex=True, inplace=True)

##df2: Coluna dt_acidente2

In [ ]:
df2['dt_acidente2'].value_counts()

01/06/2022    2087
06/06/2022    2060
08/06/2022    2033
07/06/2022    2011
02/06/2022    1980
              ... 
08/05/2022      81
27/03/2022      79
27/02/2022      70
17/04/2022      67
02/01/2022      63
Name: dt_acidente2, Length: 181, dtype: int64

##df2: Coluna dt_nasc

In [ ]:
df2['dt_nasc'].value_counts()

00/00/0000    192
26/08/1991     18
12/02/1987     18
07/07/1981     18
07/09/1981     18
             ... 
10/07/1955      1
03/08/2001      1
12/02/1965      1
17/03/1960      1
08/11/1965      1
Name: dt_nasc, Length: 16925, dtype: int64

In [ ]:
#Valor 00/00/0000 foi considerado inconsistencia, mudamos pra NA
df2['dt_nasc'].replace(['00/00/0000'], pd.NA, regex=True, inplace=True)

##df2: Coluna dt_emissao_cat

In [ ]:
df2['dt_emissao_cat'].value_counts()

00/00/0000    39913
28/06/2022     2616
27/06/2022     2515
21/06/2022     2510
14/06/2022     2428
10/06/2022     2428
23/06/2022     2397
22/06/2022     2373
29/06/2022     2363
15/06/2022     2363
13/06/2022     2354
30/06/2022     2353
09/06/2022     2338
07/06/2022     2337
08/06/2022     2324
20/06/2022     2298
03/06/2022     2250
06/06/2022     2241
01/06/2022     2191
24/06/2022     2186
02/06/2022     2144
17/06/2022     1600
16/06/2022      353
11/06/2022      157
25/06/2022      144
04/06/2022      144
18/06/2022      111
05/06/2022       52
12/06/2022       45
26/06/2022       42
19/06/2022       32
Name: dt_emissao_cat, dtype: int64

In [ ]:
#Valor 00/00/0000 foi considerado inconsistencia, mudamos pra NA
df2['dt_emissao_cat'].replace(['00/00/0000'], pd.NA, regex=True, inplace=True)

##df2: Coluna cnpj_cei

In [ ]:
df2['cnpj_cei'].value_counts()

00.000.000.000.000    78611
57.571.275.002.570      286
47.673.793.010.217       99
28.549.483.000.105       98
46.319.000.000.150       65
                      ...  
25.092.570.000.180        1
32.273.055.000.161        1
14.694.138.000.142        1
04.994.418.000.112        1
00.030.808.740.210        1
Name: cnpj_cei, Length: 7278, dtype: int64

In [ ]:
#Valor 00.000.000.000.000 foi considerado inconsistencia, mudamos pra NA
df2['cnpj_cei'].replace(['00.000.000.000.000'], pd.NA, regex=True, inplace=True)

##df2: Coluna cod_uf

In [ ]:
df2['cod_uf'].value_counts()

000000    41986
355030     4479
330455     1450
310620      984
431490      895
          ...  
411722        1
261260        1
110094        1
410090        1
431053        1
Name: cod_uf, Length: 2380, dtype: int64

In [ ]:
#Valor 000000 foi considerado inconsistencia, mudamos pra NA
df2['cod_uf'].replace(['000000'], pd.NA, regex=True, inplace=True)

#REORDENANDO: df2

In [ ]:
df2 = df2[['dt_acidente','dt_acidente2','dt_despacho','mes_afastamento','dt_emissao_cat','sexo','dt_nasc','uf_empregador','uf_acidente','cod_uf','municipio','filiacao','beneficio','cat','tipo_acidente','agt_causador','lesao','local_lesao','obito','cid_cod','cid_desc','cbo_cod','cbo_desc','cnae_cod','cnae_desc','cnpj_cei']]

In [ ]:
#Percebemos algumas datas redundantes, como por exemplo a coluna de dt_acidente e dt_acidente2, decidimos dropar a menos específica.
df2.drop(['dt_acidente'], axis=1, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
#renomeamos a coluna dt_acidente2 para dt_acidente
df2.rename(columns={'dt_acidente2':'dt_acidente'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
df2 = df2.fillna(np.nan)

In [ ]:
#ultima remoção de possíveis espaços em branco
remove_espaco(df2)

In [ ]:
df2.head(2)

,dt_acidente,dt_despacho,mes_afastamento,dt_emissao_cat,sexo,dt_nasc,uf_empregador,uf_acidente,cod_uf,municipio,filiacao,beneficio,cat,tipo_acidente,agt_causador,lesao,local_lesao,obito,cid_cod,cid_desc,cbo_cod,cbo_desc,cnae_cod,cnae_desc,cnpj_cei
0,26/05/2022,NaN,2022/05,01/06/2022,Masculino,13/11/1989,São Paulo,Maranhão,354780,Santo André Sp,Empregado,Pa,Empregador,Típico,NaN,Lesao Imediata,Dedo,Não,Y28.2,Escolas Outr Instit Areas Admin Publica,322205,Tec. de Enfermagem,8630.0,Atividades de Atencao Ambulatorial Executadas,57.571.275.002.570
1,26/05/2022,NaN,2022/05,01/06/2022,Feminino,19/04/1990,São Paulo,Maranhão,354780,Santo André Sp,Empregado,Pa,Empregador,Trajeto,Rua e Estrada Superficie Utilizada para Sus,Luxacao,Perna (Do Tornozelo Exclusive ao Joelho Ex,Não,S93.4,Entorse e Distensao do Tornozelo,992225,Auxiliar Geral de Conservação de Vias,8630.0,Atividades de Atencao Ambulatorial Executadas,57.571.275.002.570


In [ ]:
#Drop de linhas duplicadas
df2.drop_duplicates(inplace=True)

#TRATAMENTO INICIAL: df3

In [ ]:
#Analisando o DataFrame
df3

,Agente Causador Acidente,Data Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,Munic Empr,Natureza da Lesão,Origem de Cadastramento CAT,Parte Corpo Atingida,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,"CNPJ/CEI Empregador,,,,,,,,,,"
0,"Faca,Facao- Ferramenta Manual sem Forca Motr",2022/10,848515-Desossador,S51.0 Ferim do Cotovelo,1011,"Abate de Reses,Exceto Suinos",Empregador,Pa,Empregado,Não,521570-Palmeiras de Goiás,"Corte,Laceracao,Ferida Contusa,Punctura",Internet,Cotovelo,Masculino,Típico,{ñ class},Goiás,2022/10,0000/00,05/10/2022,05/11/1988,06/10/2022,"00.000.000.000.000,,,,,"
1,"Aprision. Em,Sob ou Entre,Nic",2022/10,324115-Tec. em Radiologia e Imagenologia,S60.0 Contusao de Dedos s/Lesao da Unha,8610,Atividades de Atendimento Hospitalar,Empregador,Pa,Empregado,Não,352590-Jundiaí,"Contusao,Esmagamento (Superficie Cutanea I",Internet,Dedo,Masculino,Típico,Maranhão,São Paulo,2022/10,0000/00,04/10/2022,02/02/1989,06/10/2022,"00.000.000.000.000,,,,,,,"
2,"Andaime,Plataforma - Edificio ou Estrutura",2022/10,715210-Pedreiro,S62.8 Frat de Outr Partes e de Partes Ne Punh,4120,Construcao de Edificios,Empregador,Pa,Empregado,Não,421370-Pouso Redondo,Fratura,Internet,Mao (Exceto Punho ou Dedos),Masculino,Típico,{ñ class},Santa Catarina,2022/10,0000/00,05/10/2022,07/02/1981,06/10/2022,"00.000.000.000.000,,,,,,,,,"
3,Veiculo Rodoviario Motorizado,2022/10,784205-Alimentador de Linha de Prod.,T02.2 Frat Envolv Regioes Mult de Um Membro S,4211,Construcao de Rodovias e Ferrovias,Empregador,Pa,Empregado,Sim,314480-Nova Lima,Lesoes Multiplas,Internet,Partes Multiplas - Aplica-Se Quando Mais de U,Masculino,Típico,Rondônia,Minas Gerais,2022/10,0000/00,05/10/2022,27/09/1958,06/10/2022,"00.000.000.000.000,,,,,,,,,,"
4,Impacto de Pes. Contra Objeto em Movimento,2022/10,717020-Servente de Obras,S62.5 Frat do Polegar,4110,Incorporacao de Empreendimentos Imobiliarios,Empregador,Pa,Empregado,Não,292740-Salvador,Fratura,Internet,Dedo,Masculino,Típico,{ñ class},Bahia,2022/10,0000/00,05/10/2022,09/08/1976,06/10/2022,"00.000.000.000.000,,,,,,,,,,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15654,Piso de Edificio - Superficie Utilizada Para,2022/09,{ñ class},S62 Frat ao Nivel do Punho e da Mao,2330,"Fabricacao de Artefatos de Concreto,Cimento,",{ñ class},Auxílio Doenca por Acidente do Trabalho,Empregado,Não,410960-Guaratuba,Lesao Imediata,Internet,Mao (Exceto Punho ou Dedos),Masculino,Típico,Roraima,Paraná,2022/09,2022/10,29/09/2022,18/09/1993,10/10/2022,"21.161.901.000.135,,,,,,,,"
15655,Escada Permanente Cujos Degraus Permitem Apoi,2022/10,232130-Prof. Física no Ensino Médio,S82 Frat da Perna Incl Tornozelo,8520,Ensino Medio,Autoridade Pública,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,410690-Curitiba,Fratura,Internet,"Perna (Do Tornozelo,Exclusive,ao Joelho,Ex",Feminino,Típico,Roraima,Paraná,2022/10,2022/11,06/10/2022,24/11/1986,07/10/2022,"76.416.965.000.121,,,,,,,"
15656,"Andaime,Plataforma - Edificio ou Estrutura",2022/09,517330-Vigilante,S82.7 Frat Mult da Perna,8112,Condominios Prediais,Segurado/Dependente,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,420460-Criciúma,Fratura,Internet,"Membros Inferiores,Partes Multiplas (Qualque",Masculino,Típico,{ñ class},Santa Catarina,2022/09,2022/10,20/09/2022,02/03/1986,10/10/2022,"32.994.401.000.109,,,,,,,,"
15657,"Martelo,Malho,Marreta- Ferramenta Manual Se",2022/10,724315-Soldador,S62.6 Frat de Outr Dedos,7112,Servicos de Engenharia,Empregador,Auxílio Doenca por Acidente do Trabalho,Empregado,Não,411000-Iguaraçu,Fratura,Internet,Dedo,Masculino,Típico,Roraima,Paraná,2022/10,2022/10,05/10/2022,08/04/1984,06/10/2022,"34.197.591.000.150,,,,,,,,"


In [ ]:
#Renomeando Colunas
df3.dtypes

Agente  Causador  Acidente       object
Data Acidente                    object
CBO                              object
CID-10                           object
CNAE2.0 Empregador                int64
CNAE2.0 Empregador.1             object
Emitente CAT                     object
Espécie do benefício             object
Filiação Segurado                object
Indica Óbito Acidente            object
Munic Empr                       object
Natureza da Lesão                object
Origem de Cadastramento CAT      object
Parte Corpo Atingida             object
Sexo                             object
Tipo do Acidente                 object
UF  Munic.  Acidente             object
UF Munic. Empregador             object
Data Acidente.1                  object
Data Despacho Benefício          object
Data Acidente.2                  object
Data Nascimento                  object
Data Emissão CAT                 object
CNPJ/CEI Empregador,,,,,,,,,,    object
dtype: object

In [ ]:
df3.rename(columns={'CBO':'cbo',
                    'Agente  Causador  Acidente':'agt_causador',
                    'Data Acidente':'dt_acidente',
                    'CID-10':'cid',
                    'CNAE2.0 Empregador':'cnae_cod',
                    'CNAE2.0 Empregador.1':'cnae_desc',
                    'Emitente CAT':'cat',
                    'Espécie do benefício':'beneficio',
                    'Filiação Segurado':'filiacao',
                    'Indica Óbito Acidente':'obito',
                    'Munic Empr':'municipio',
                    'Natureza da Lesão':'lesao',
                    'Origem de Cadastramento CAT':'origem_cad',
                    'Parte Corpo Atingida':'local_lesao',
                    'Sexo':'sexo',
                    'Tipo do Acidente':'tipo_acidente',
                    'UF  Munic.  Acidente':'uf_acidente',
                    'UF Munic. Empregador':'uf_empregador',
                    'Data Acidente.1':'mes_afastamento',
                    'Data Despacho Benefício':'dt_despacho',
                    'Data Acidente.2':'dt_acidente2',
                    'Data Nascimento':'dt_nasc',
                    'Data Emissão CAT':'dt_emissao_cat',
                    'CNPJ/CEI Empregador,,,,,,,,,,':'cnpj_cei'
                    }, inplace=True)

In [ ]:
#verificando se ficou tudo ok
df3.dtypes

agt_causador       object
dt_acidente        object
cbo                object
cid                object
cnae_cod            int64
cnae_desc          object
cat                object
beneficio          object
filiacao           object
obito              object
municipio          object
lesao              object
origem_cad         object
local_lesao        object
sexo               object
tipo_acidente      object
uf_acidente        object
uf_empregador      object
mes_afastamento    object
dt_despacho        object
dt_acidente2       object
dt_nasc            object
dt_emissao_cat     object
cnpj_cei           object
dtype: object

In [ ]:
#Ajustando valores inconsistentes de primeira vista no df
df3.replace([',','-'],' ',regex=True, inplace=True)

In [ ]:
#Substituindo valores inconsistentes de primeira vista no df por <NA>
df3.replace(['{ñ class}', '{ñ class', '{ñ'],pd.NA,regex=True, inplace=True)

In [ ]:
#Removendo os espaços em branco
#Aplicando a função removedora de espaços em branco
remove_espaco(df3)

In [ ]:
#Verificando se ficou tudo ok
df3.head()

,agt_causador,dt_acidente,cbo,cid,cnae_cod,cnae_desc,cat,beneficio,filiacao,obito,municipio,lesao,origem_cad,local_lesao,sexo,tipo_acidente,uf_acidente,uf_empregador,mes_afastamento,dt_despacho,dt_acidente2,dt_nasc,dt_emissao_cat,cnpj_cei
0,Faca Facao Ferramenta Manual sem Forca Motr,2022/10,848515 Desossador,S51.0 Ferim do Cotovelo,1011,Abate de Reses Exceto Suinos,Empregador,Pa,Empregado,Não,521570 Palmeiras de Goiás,Corte Laceracao Ferida Contusa Punctura,Internet,Cotovelo,Masculino,Típico,<NA>,Goiás,2022/10,0000/00,05/10/2022,05/11/1988,06/10/2022,00.000.000.000.000
1,Aprision. Em Sob ou Entre Nic,2022/10,324115 Tec. em Radiologia e Imagenologia,S60.0 Contusao de Dedos s/Lesao da Unha,8610,Atividades de Atendimento Hospitalar,Empregador,Pa,Empregado,Não,352590 Jundiaí,Contusao Esmagamento (Superficie Cutanea I,Internet,Dedo,Masculino,Típico,Maranhão,São Paulo,2022/10,0000/00,04/10/2022,02/02/1989,06/10/2022,00.000.000.000.000
2,Andaime Plataforma Edificio ou Estrutura,2022/10,715210 Pedreiro,S62.8 Frat de Outr Partes e de Partes Ne Punh,4120,Construcao de Edificios,Empregador,Pa,Empregado,Não,421370 Pouso Redondo,Fratura,Internet,Mao (Exceto Punho ou Dedos),Masculino,Típico,<NA>,Santa Catarina,2022/10,0000/00,05/10/2022,07/02/1981,06/10/2022,00.000.000.000.000
3,Veiculo Rodoviario Motorizado,2022/10,784205 Alimentador de Linha de Prod.,T02.2 Frat Envolv Regioes Mult de Um Membro S,4211,Construcao de Rodovias e Ferrovias,Empregador,Pa,Empregado,Sim,314480 Nova Lima,Lesoes Multiplas,Internet,Partes Multiplas Aplica Se Quando Mais de U,Masculino,Típico,Rondônia,Minas Gerais,2022/10,0000/00,05/10/2022,27/09/1958,06/10/2022,00.000.000.000.000
4,Impacto de Pes. Contra Objeto em Movimento,2022/10,717020 Servente de Obras,S62.5 Frat do Polegar,4110,Incorporacao de Empreendimentos Imobiliarios,Empregador,Pa,Empregado,Não,292740 Salvador,Fratura,Internet,Dedo,Masculino,Típico,<NA>,Bahia,2022/10,0000/00,05/10/2022,09/08/1976,06/10/2022,00.000.000.000.000


#TRATAMENTO DE COLUNAS: df3

##df3: Coluna agt_causador

In [ ]:
df3['agt_causador'].value_counts()

Impacto de Pes. Contra Objeto em Movimento       948
Impacto de Pes. Contra Objeto Parado             689
Motocicleta Motoneta                             677
Impacto Sofrido por Pes. de Objeto Que Cai       583
Queda de Pes. com Dif. de Nivel de Vei           512
                                                ... 
Equip. de Iluminacao                               1
Temperatura Ambiente  nao Inclui a de Objeto       1
Elevador de Cacamba para Mineracao   Equipame      1
Couro Cru ou Curtido   Produto Animal              1
Pua Trado Verruma Maquina de Furar Manual          1
Name: agt_causador, Length: 258, dtype: int64

##df3: Coluna dt_acidente

In [ ]:
df3['dt_acidente'].value_counts()

2022/10    11321
2022/09     3317
2022/08      406
2022/07      197
2022/06      110
2022/05       78
2022/01       73
2022/04       57
2022/02       51
2022/03       49
Name: dt_acidente, dtype: int64

##df3: Coluna cbo

In [ ]:
df3['cbo'].value_counts()

322205 Tec. de Enfermagem                        1057
784205 Alimentador de Linha de Prod.             1026
717020 Servente de Obras                          382
782510 Motorista de Caminhão (Rotas Regionais     358
223505 Enfermeiro                                 236
                                                 ... 
761351 Oper. de Espuladeira                         1
841710 Filtrador de Cerveja                         1
510135 Maître                                       1
371210 Tec. em Museologia                           1
763310 Bordador à Máquina                           1
Name: cbo, Length: 976, dtype: int64

In [ ]:
#separando o código do texto da coluna cbo (Código Brasileiro de Ocupação)
df3['cbo_cod'] = df3['cbo'].str.slice(0, 7)

In [ ]:
#Renomeando a coluna cbo para cbo_desc ('cbo descrição')
df3.rename(columns={'cbo':'cbo_desc'}, inplace=True)

##df3: Coluna cbo_cod

In [ ]:
df3['cbo_cod'].value_counts()

322205     1057
784205     1026
717020      382
782510      358
223505      236
           ... 
761351        1
841710        1
510135        1
371210        1
763310        1
Name: cbo_cod, Length: 976, dtype: int64

In [ ]:
#Valores '0000000' foram considerados inconsistencias, mudamos pra NA
df3['cbo_cod'].replace(['000000'], pd.NA, regex=True, inplace=True)

##df3: Coluna cbo_desc

In [ ]:
df3['cbo_desc'].value_counts()

322205 Tec. de Enfermagem                        1057
784205 Alimentador de Linha de Prod.             1026
717020 Servente de Obras                          382
782510 Motorista de Caminhão (Rotas Regionais     358
223505 Enfermeiro                                 236
                                                 ... 
761351 Oper. de Espuladeira                         1
841710 Filtrador de Cerveja                         1
510135 Maître                                       1
371210 Tec. em Museologia                           1
763310 Bordador à Máquina                           1
Name: cbo_desc, Length: 976, dtype: int64

In [ ]:
#Valores 'Não informado' foram considerados inconsistencias, mudamos para NA
df3['cbo_desc'].replace(['Não Informado'], pd.NA, regex=True, inplace=True)

##df3: Coluna cid

In [ ]:
df3['cid'].value_counts()

S61.0 Ferim de Dedos s/Lesao da Unha             810
S61.1 Ferim de Dedos c/Lesao da Unha             679
S93.4 Entorse e Distensao do Tornozelo           605
S62.6 Frat de Outr Dedos                         459
S80.0 Contusao do Joelho                         415
                                                ... 
H01.0 Blefarite                                    1
S21.2 Ferim da Parede Posterior do Torax           1
Z09.8 Exame Seguim Apos Outr Trat p/Outr Afec      1
S87.8 Traum p/Esmag Outr Parte e Partes Ne Pe      1
S82   Frat da Perna Incl Tornozelo                 1
Name: cid, Length: 1130, dtype: int64

In [ ]:
#Separando os valores numéricos dos valores escritos da coluna cid
df3['cod_cid'] = df3['cid'].str.slice(0,6)

In [ ]:
#removendo os números da coluna cid
df3['cid'] = df3['cid'].str.replace(r'\d+','', regex=True)

In [ ]:
#Renomenaod as colunas para melhor entendimento (cid_desc = cid descrição e cid_cod = cid código)
df3.rename(columns={'cid':'cid_desc', 'cod_cid':'cid_cod'}, inplace=True)

##df3: Coluna cid_cod

In [ ]:
df3['cid_cod'].value_counts()

S61.0     810
S61.1     679
S93.4     605
S62.6     459
S80.0     415
         ... 
H01.0       1
S21.2       1
Z09.8       1
S87.8       1
S82         1
Name: cid_cod, Length: 1130, dtype: int64

In [ ]:
#Retirando espaços em branco
df3['cid_cod'] = df3['cid_cod'].str.strip()

##df3: Coluna cid_desc

In [ ]:
df3['cid_desc'].value_counts()

S. Ferim de Dedos s/Lesao da Unha             810
S. Ferim de Dedos c/Lesao da Unha             679
S. Entorse e Distensao do Tornozelo           605
S. Frat de Outr Dedos                         459
S. Contusao do Joelho                         415
                                             ... 
F. Transt Ne da Personalidade                   1
L. Outr Afec Granulomatosas Pele Tec Subcu      1
H. Blefarite                                    1
S. Ferim da Parede Posterior do Torax           1
S   Frat da Perna Incl Tornozelo                1
Name: cid_desc, Length: 917, dtype: int64

In [ ]:
#Removendo os valores não numéricos do código CID
df3['cid_desc'] = df3['cid_desc'].str.replace(r'^..','', regex=True)

In [ ]:
#Retirando os espaços que sobraram
df3['cid_desc'] = df3['cid_desc'].str.strip()

##df3: Coluna cnae_cod

In [ ]:
df3['cnae_cod'].value_counts()

8610    1532
4711     596
8411     471
0        448
4930     436
        ... 
5020       1
5914       1
1750       1
114        1
6511       1
Name: cnae_cod, Length: 640, dtype: int64

In [ ]:
#O valor '0' foi considerado inconsistencia, substituímos por NA.
df3['cnae_cod'].replace([0],pd.NA,regex=True, inplace=True)

##df3: Coluna cnae_desc

In [ ]:
df3['cnae_desc'].value_counts()

Atividades de Atendimento Hospitalar             1532
Comercio Varejista de Mercadorias em Geral c      661
Administracao Publica em Geral                    471
Transporte Rodoviario de Carga                    436
Construcao de Edificios                           359
                                                 ... 
Fabricacao de Acessorios do Vestuario Exceto        1
Pesca em Agua Doce                                  1
Gestao de Ativos Intangiveis Nao Financeiros        1
Fabricacao de Artefatos de Couro nao Especifi       1
Seguros de Vida                                     1
Name: cnae_desc, Length: 546, dtype: int64

##df3: Coluna cat

In [ ]:
df3['cat'].value_counts()

Empregador             14635
Segurado/Dependente      473
Sindicato                193
Autoridade Pública       124
Médico                   102
Name: cat, dtype: int64

##df3: Coluna beneficio

In [ ]:
df3['beneficio'].value_counts()

Pa                                         15650
Auxílio Doenca por Acidente do Trabalho        9
Name: beneficio, dtype: int64

##df3: Coluna filiacao

In [ ]:
df3['filiacao'].value_counts()

Empregado             15507
Trabalhador Avulso       56
Segurado Especial        13
Name: filiacao, dtype: int64

##df3: Coluna obito

In [ ]:
df3['obito'].value_counts()

Não    15557
Sim      102
Name: obito, dtype: int64

##df3: Coluna municipio

In [ ]:
df3['municipio'].value_counts()

355030 São Paulo                  1461
000000 Ignorado                    755
330455 Rio de Janeiro              533
310620 Belo Horizonte              333
410690 Curitiba                    258
                                  ... 
420768 Ipuaçu                        1
292600 Remanso                       1
431730 Santa Vitória do Palmar       1
314410 Muzambinho                    1
311280 Capitólio                     1
Name: municipio, Length: 1622, dtype: int64

In [ ]:
#Separando o código do município do nome para melhor avaliação dos dados e criando uma nova coluna cod_uf para os códigos
df3['cod_uf'] = df3['municipio'].str.slice(0, 6)

In [ ]:
#Apagando o código dos municípios na coluna municipio
df3['municipio'] = df3['municipio'].str.replace(r'\d+','', regex=True)

In [ ]:
#valor "Ignorado" é uma inconsistencia, será substituído por NA.
df3['municipio'].replace(['Ignorado'],pd.NA,regex=True, inplace=True)

##df3: Coluna lesao

In [ ]:
df3['lesao'].value_counts()

Fratura                                        2797
Corte Laceracao Ferida Contusa Punctura        2698
Lesao Imediata                                 1851
Contusao Esmagamento (Superficie Cutanea I     1680
Escoriacao Abrasao (Ferimento Superficial)     1408
Distensao Torcao                               1232
Outras Lesoes Nic                               952
Luxacao                                         874
Lesao Imediata Nic                              463
Queimadura ou Escaldadura   Efeito de Tempe     419
Lesoes Multiplas                                301
Doenca Nic                                      192
Inflamacao de Articulacao Tendao ou Muscul      155
Doenca Contagiosa ou Infecciosa (Tuberculos     152
Amputacao ou Enucleacao                         118
Queimadura Quimica (Lesao de Tecido Provoca     109
Concussao Cerebral                               55
Envenenamento Sistemico   Condicao Morbida       51
Choque Eletrico e Eletroplessao (Eletrocuss      46
Hernia de Qu

##df3: Coluna origem_cad

In [ ]:
df3['origem_cad'].value_counts()

Internet    15659
Name: origem_cad, dtype: int64

In [ ]:
#Coluna com valor constante. Optamos por drop
df3.drop(['origem_cad'], axis=1, inplace=True)

##df3: Coluna local_lesao

In [ ]:
df3['local_lesao'].value_counts()

Dedo                                             3538
Pe (Exceto Artelhos)                             1343
Mao (Exceto Punho ou Dedos)                      1181
Joelho                                            866
Partes Multiplas   Aplica Se Quando Mais de U     696
Perna (Entre O Tornozelo e a Pelvis)              685
Braco (Entre O Punho a O Ombro)                   628
Articulacao do Tornozelo                          583
Ombro                                             543
Perna (Do Tornozelo Exclusive ao Joelho Ex        538
Olho (Inclusive Nervo Otico e Visao)              506
Punho                                             412
Cabeca Nic                                        383
Dorso (Inclusive Musculos Dorsais Coluna e M      376
Antebraco (Entre O Punho e O Cotovelo)            333
Membros Superiores Partes Multiplas (Qualque      315
Face Partes Multiplas (Qualquer Combinacao D      229
Cabeca Partes Multiplas (Qualquer Combinacao      227
Membros Inferiores Nic      

##df3: Coluna sexo

In [ ]:
df3['sexo'].value_counts()

Masculino        10118
Feminino          5491
Não Informado       50
Name: sexo, dtype: int64

In [ ]:
#Valores não informados consideramos como inconsistencias e mudamos por NA
df3['sexo'].replace(['Não Informado'], pd.NA, regex=True, inplace=True)

##df3: Coluna tipo_acidente

In [ ]:
df3['tipo_acidente'].value_counts()

Típico     11790
Trajeto     3456
Doença       413
Name: tipo_acidente, dtype: int64

##df3: Coluna uf_acidente

In [ ]:
df3['uf_acidente'].value_counts()

Maranhão            5473
Rondônia            1546
Roraima             1198
Tocantins           1000
Zerado               748
Pará                 300
Ceará                211
Acre                 211
Pernambuco            76
Amazonas              74
Sergipe               61
Amapá                 44
Piauí                 41
Alagoas               18
Rio Grande Norte      14
Paraíba                9
Name: uf_acidente, dtype: int64

In [ ]:
#Entendemos Zerado como uma inconsistencia, mudamos pra NA
df3['uf_acidente'].replace(['Zerado'], pd.NA, regex=True, inplace=True)

##df3: Coluna uf_empregador

In [ ]:
df3['uf_empregador'].value_counts()

São Paulo              5473
Minas Gerais           1546
Paraná                 1198
Rio Grande do Sul      1179
Santa Catarina         1144
Rio de Janeiro         1000
Zerado                  748
Bahia                   403
Goiás                   401
Espírito Santo          327
Ceará                   308
Pernambuco              300
Mato Grosso             248
Distrito Federal        211
Pará                    211
Mato Grosso do Sul      177
Amazonas                163
Alagoas                 105
Rio Grande do Norte      96
Maranhão                 84
Rondônia                 76
Paraíba                  74
Tocantins                61
Piauí                    44
Sergipe                  41
Roraima                  18
Acre                     14
Amapá                     9
Name: uf_empregador, dtype: int64

In [ ]:
#Entendemos Zerado como uma inconsistencia, mudamos pra NA
df3['uf_empregador'].replace(['Zerado'], pd.NA, regex=True, inplace=True)

##df3: Coluna mes_afastamento

In [ ]:
df3['mes_afastamento'].value_counts()

2022/10    11321
2022/09     3317
2022/08      406
2022/07      197
2022/06      110
2022/05       78
2022/01       73
2022/04       57
2022/02       51
2022/03       49
Name: mes_afastamento, dtype: int64

##df3: Coluna dt_despacho

In [ ]:
df3['dt_despacho'].value_counts()

0000/00    15650
2022/10        7
2022/11        2
Name: dt_despacho, dtype: int64

In [ ]:
#Valor 0000/00 foi considerado inconsistencia, mudamos pra NA
df3['dt_despacho'].replace(['0000/00'], pd.NA, regex=True, inplace=True)

##df3: Coluna dt_acidente2

In [ ]:
df3['dt_acidente2'].value_counts()

03/10/2022    883
04/10/2022    844
05/10/2022    838
06/10/2022    798
10/10/2022    733
             ... 
11/04/2022      1
25/02/2022      1
01/03/2022      1
26/02/2022      1
26/03/2022      1
Name: dt_acidente2, Length: 272, dtype: int64

##df3: Coluna dt_nasc

In [ ]:
df3['dt_nasc'].value_counts()

28/03/1985    14
05/01/1973    13
23/09/2000    12
23/03/1977    11
28/09/1971    11
              ..
08/09/1981     1
26/04/1961     1
15/03/1989     1
06/09/1996     1
24/11/1986     1
Name: dt_nasc, Length: 9538, dtype: int64

In [ ]:
#Valor 00/00/0000 foi considerado inconsistencia, mudamos pra NA
df3['dt_nasc'].replace(['00/00/0000'], pd.NA, regex=True, inplace=True)

##df3: Coluna dt_emissao_cat

In [ ]:
df3['dt_emissao_cat'].value_counts()

06/10/2022    1282
11/10/2022    1158
04/10/2022    1157
07/10/2022    1113
05/10/2022    1075
03/10/2022    1050
13/10/2022     943
10/10/2022     907
18/10/2022     802
14/10/2022     799
19/10/2022     739
17/10/2022     737
20/10/2022     577
21/10/2022     511
26/10/2022     509
27/10/2022     496
24/10/2022     489
25/10/2022     451
28/10/2022     286
31/10/2022     235
12/10/2022      87
01/10/2022      79
08/10/2022      44
15/10/2022      36
09/10/2022      20
22/10/2022      19
29/10/2022      19
23/10/2022      14
16/10/2022      11
02/10/2022       9
30/10/2022       5
Name: dt_emissao_cat, dtype: int64

##df3: Coluna cnpj_cei

In [ ]:
df3['cnpj_cei'].value_counts()

00.000.000.000.000    14367
60.701.190.000.104       15
45.276.128.000.110       13
57.571.275.002.570       11
19.876.424.000.142        9
                      ...  
28.369.690.000.170        1
04.948.916.000.129        1
08.218.991.000.195        1
12.397.073.000.148        1
54.614.813.000.109        1
Name: cnpj_cei, Length: 1016, dtype: int64

In [ ]:
#Valor 00.000.000.000.000 foi considerado inconsistencia, mudamos pra NA
df3['cnpj_cei'].replace(['00.000.000.000.000'], pd.NA, regex=True, inplace=True)

##df3: Coluna cod_uf

In [ ]:
df3['cod_uf'].value_counts()

355030    1461
000000     755
330455     533
310620     333
410690     258
          ... 
420768       1
292600       1
431730       1
314410       1
311280       1
Name: cod_uf, Length: 1622, dtype: int64

In [ ]:
#Valor 000000 foi considerado inconsistencia, mudamos pra NA
df3['cod_uf'].replace(['000000'], pd.NA, regex=True, inplace=True)

#REORDENANDO: df3

In [ ]:
df3 = df3[['dt_acidente','dt_acidente2','dt_despacho','mes_afastamento','dt_emissao_cat','sexo','dt_nasc','uf_empregador','uf_acidente','cod_uf','municipio','filiacao','beneficio','cat','tipo_acidente','agt_causador','lesao','local_lesao','obito','cid_cod','cid_desc','cbo_cod','cbo_desc','cnae_cod','cnae_desc','cnpj_cei']]

In [ ]:
#Percebemos algumas datas redundantes, como por exemplo a coluna de dt_acidente e dt_acidente2, decidimos dropar a menos específica.
df3.drop(['dt_acidente'], axis=1, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
#renomeamos a coluna dt_acidente2 para dt_acidente
df3.rename(columns={'dt_acidente2':'dt_acidente'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
df3 = df3.fillna(np.nan)

In [ ]:
#ultima remoção de possíveis espaços em branco
remove_espaco(df3)

In [ ]:
df3.head(2)

,dt_acidente,dt_despacho,mes_afastamento,dt_emissao_cat,sexo,dt_nasc,uf_empregador,uf_acidente,cod_uf,municipio,filiacao,beneficio,cat,tipo_acidente,agt_causador,lesao,local_lesao,obito,cid_cod,cid_desc,cbo_cod,cbo_desc,cnae_cod,cnae_desc,cnpj_cei
0,05/10/2022,NaN,2022/10,06/10/2022,Masculino,05/11/1988,Goiás,NaN,521570,Palmeiras de Goiás,Empregado,Pa,Empregador,Típico,Faca Facao Ferramenta Manual sem Forca Motr,Corte Laceracao Ferida Contusa Punctura,Cotovelo,Não,S51.0,Ferim do Cotovelo,848515,848515 Desossador,1011.0,Abate de Reses Exceto Suinos,NaN
1,04/10/2022,NaN,2022/10,06/10/2022,Masculino,02/02/1989,São Paulo,Maranhão,352590,Jundiaí,Empregado,Pa,Empregador,Típico,Aprision. Em Sob ou Entre Nic,Contusao Esmagamento (Superficie Cutanea I,Dedo,Não,S60.0,Contusao de Dedos s/Lesao da Unha,324115,324115 Tec. em Radiologia e Imagenologia,8610.0,Atividades de Atendimento Hospitalar,NaN


In [ ]:
#Drop de linhas duplicadas
df3.drop_duplicates(inplace=True)

#CONCATENAÇÃO DOS 3 DATAFRAMES

In [ ]:
#Novo DataFrame com os valores dos 3 DataFrames juntos concatenados em axis = 0 para que se adicionem as linhas e não colunas.
df4 = pd.concat([df, df2, df3], axis=0)

In [ ]:
#Valor final de linhas ficou como o esperado e o de colunas se manteve constante.
df4

In [ ]:
df4.dtypes

In [ ]:
remove_espaco(df4)

In [ ]:
df4.drop_duplicates()

#VALIDAÇÃO DE DADOS

In [ ]:
schema = pa.DataFrameSchema(
      columns = {
          'dt_acidente':pa.Column(pa.String),
          'dt_despacho':pa.Column(pa.String, nullable=True),
          'mes_afastamento':pa.Column(pa.String),
          'dt_emissao_cat':pa.Column(pa.String, nullable=True),
          'sexo':pa.Column(pa.String, nullable=True),
          'dt_nasc':pa.Column(pa.String, nullable=True),
          'uf_empregador':pa.Column(pa.String, nullable=True),
          'uf_acidente':pa.Column(pa.String, nullable=True),
          'cod_uf':pa.Column(pa.String, nullable=True),
          'municipio':pa.Column(pa.String, nullable=True),
          'filiacao':pa.Column(pa.String, nullable=True),
          'beneficio':pa.Column(pa.String),
          'cat':pa.Column(pa.String, nullable=True),
          'tipo_acidente':pa.Column(pa.String),
          'agt_causador':pa.Column(pa.String, nullable=True),
          'lesao':pa.Column(pa.String, nullable=True),
          'local_lesao':pa.Column(pa.String, nullable=True),
          'obito':pa.Column(pa.String, nullable=True),
          'cid_cod':pa.Column(pa.String, nullable=True),
          'cid_desc':pa.Column(pa.String, nullable=True),
          'cbo_cod':pa.Column(pa.String, nullable=True),
          'cbo_desc':pa.Column(pa.String, nullable=True),
          'cnae_cod':pa.Column(pa.Float, nullable=True),
          'cnae_desc':pa.Column(pa.String, nullable=True),
          'cnpj_cei':pa.Column(pa.String, nullable=True),
      }
  )

In [ ]:
schema.validate(df4)

#ENVIANDO OS DFs TRATADOS: mongoDB + gcloud

#☁ gCloud

In [ ]:
#Instalação e importação da biblioteca gcloud storage
!pip install gcsfs
from google.cloud import storage
import os

In [ ]:
#Configuração da chave de segurança json da serviceaccount
serviceAccount = '/content/aulas-bc26-gabriela-c46064bcf2f2.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#Criando os novos df's

df_tratado = df
df2_tratado = df2
df3_tratado = df3
df4_tratado = df4

In [ ]:
#ENVIANDO O PRIMEIRO DF TRATADO PARA O BUCKET
bucket_name = 'bc26-gabriela-bucket'

# tansformando o df em .csv
df_tratado.to_csv('df_tratado.csv', index = False)
# conectando com o storage client
client = storage.Client.from_service_account_json('/content/aulas-bc26-gabriela-c46064bcf2f2.json')
# buscando o bucket
bucket = client.bucket(bucket_name)
# criando blob (onde vc quer enviar o arquivo .csv)
blob = bucket.blob("projeto_aquecimento/df_tratado.csv")
# enviando blob para o bucket
blob.upload_from_filename("df_tratado.csv")

In [ ]:
#ENVIANDO O SEGUNDO DF TRATADO PARA O BUCKET
bucket_name = 'bc26-gabriela-bucket'

# tansformando o df em .csv
df2_tratado.to_csv('df2_tratado.csv', index = False)
# conectando com o storage client
client = storage.Client.from_service_account_json('/content/aulas-bc26-gabriela-c46064bcf2f2.json')
# buscando o bucket
bucket = client.bucket(bucket_name)
# criando blob (onde vc quer enviar o arquivo .csv)
blob = bucket.blob("projeto_aquecimento/df2_tratado.csv")
# enviando blob para o bucket
blob.upload_from_filename("df2_tratado.csv")

In [ ]:
#ENVIANDO O TERCEIRO DF TRATADO PARA O BUCKET
bucket_name = 'bc26-gabriela-bucket'

# tansformando o df em .csv
df3_tratado.to_csv('df3_tratado.csv', index = False)
# conectando com o storage client
client = storage.Client.from_service_account_json('/content/aulas-bc26-gabriela-c46064bcf2f2.json')
# buscando o bucket
bucket = client.bucket(bucket_name)
# criando blob (onde vc quer enviar o arquivo .csv)
blob = bucket.blob("projeto_aquecimento/df3_tratado.csv")
# enviando blob para o bucket
blob.upload_from_filename("df3_tratado.csv")

In [ ]:
#ENVIANDO O DF CONCATENADO PARA O BUCKET
bucket_name = 'bc26-gabriela-bucket'

# tansformando o df em .csv
df4_tratado.to_csv('df4_tratado.csv', index = False)
# conectando com o storage client
client = storage.Client.from_service_account_json('/content/aulas-bc26-gabriela-c46064bcf2f2.json')
# buscando o bucket
bucket = client.bucket(bucket_name)
# criando blob (onde vc quer enviar o arquivo .csv)
blob = bucket.blob("projeto_aquecimento/df4_tratado.csv")
# enviando blob para o bucket
blob.upload_from_filename("df4_tratado.csv")

#🍃 MongoDB

In [ ]:
#Conectando com o MongoDB
uri = "mongodb+srv://cluster0.p4kvlxp.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='/content/X509-cert-5675635325549232105.pem')

In [ ]:
#Criando as coleções no mongoDB

db = client['projeto_aquecimento']
colecaotratado1 = db['tratado1']
colecaotratado2 = db['tratado2']
colecaotratado3 = db['tratado3']
colecaotratado4 = db['tratado_concatenado4']

In [ ]:
#Enviando para o mongoDB nas coleções criadas anteriormente. Aqui tivemos dificuldade pois parece não haver espaço suficiente para todos os arquivos.

'''df_tratado.reset_index(inplace=True)
df_dict_t = df_tratado.to_dict("records")
colecaotratado1.insert_many(df_dict_t)

df2_tratado.reset_index(inplace=True)
df2_dict_t = df2_tratado.to_dict("records")
colecaotratado2.insert_many(df2_dict_t)

df3_tratado.reset_index(inplace=True)
df3_dict_t = df3_tratado.to_dict("records")
colecaotratado3.insert_many(df3_dict_t)

df4_tratado.reset_index(inplace=True)
df4_dict = df4.to_dict("records")
colecaotratado4.insert_many(df4_dict)'''

'df_tratado.reset_index(inplace=True)\ndf_dict_t = df_tratado.to_dict("records")\ncolecaotratado1.insert_many(df_dict_t)\n\ndf2_tratado.reset_index(inplace=True)\ndf2_dict_t = df2_tratado.to_dict("records")\ncolecaotratado2.insert_many(df2_dict_t)\n\ndf3_tratado.reset_index(inplace=True)\ndf3_dict_t = df3_tratado.to_dict("records")\ncolecaotratado3.insert_many(df3_dict_t)\n\ndf4_tratado.reset_index(inplace=True)\ndf4_dict = df4.to_dict("records")\ncolecaotratado4.insert_many(df4_dict)'

#ANÁLISES COM SPARK E SQL

#✨ SPARK 



*   Utilizaremos o menor dataframe extraindo do bucket da gcp.




In [ ]:
#Instalando o pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 53.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=d1f23259dd4225246999ea2ed510d63469bd7ea9cfab80a1e5fb65c22c2263da
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
#Importando todos os tipos de variáveis
from pyspark.sql.types import *
#Importando a sparksession
from pyspark.sql import SparkSession
#Importando as funções como F
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [ ]:
#Configurando a sparksession com configuração adicional para conexão spark - cloud storage
spark = (
   SparkSession.builder
               .master('local')
               .appName('projeto_aquecimento')
               .config('spark.ui.port', '4050')
               .getOrCreate()                
)

In [ ]:
spark

In [ ]:
#Montando o esquema de Colunas
esquema = (
    StructType([
        StructField('dt_acidente', StringType()),
        StructField('dt_despacho', StringType()),
        StructField('mes_afastamento', StringType()),
        StructField('dt_emissao_cat', StringType()),
        StructField('sexo', StringType()),
        StructField('dt_nasc', StringType()),
        StructField('uf_empregador', StringType()),
        StructField('uf_acidente', StringType()),
        StructField('cod_uf', StringType()),
        StructField('municipio', StringType()),
        StructField('filiacao', StringType()),
        StructField('beneficio', StringType()),
        StructField('cat', StringType()),
        StructField('tipo_acidente', StringType()),
        StructField('agt_causador', StringType()),
        StructField('lesao', StringType()),
        StructField('local_lesao', StringType()),
        StructField('obito', StringType()),
        StructField('cid_cod', StringType()),
        StructField('cid_desc', StringType()),
        StructField('cbo_cod', StringType()),
        StructField('cbo_desc', StringType()),
        StructField('cnae_cod', FloatType()),
        StructField('cnae_desc', StringType()),
        StructField('cnpj_cei', StringType())
        ])
)

In [ ]:
df4_spark = spark.createDataFrame(df4, schema= esquema)

In [ ]:
df4_spark.printSchema()

root
 |-- dt_acidente: string (nullable = true)
 |-- dt_despacho: string (nullable = true)
 |-- mes_afastamento: string (nullable = true)
 |-- dt_emissao_cat: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- dt_nasc: string (nullable = true)
 |-- uf_empregador: string (nullable = true)
 |-- uf_acidente: string (nullable = true)
 |-- cod_uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- filiacao: string (nullable = true)
 |-- beneficio: string (nullable = true)
 |-- cat: string (nullable = true)
 |-- tipo_acidente: string (nullable = true)
 |-- agt_causador: string (nullable = true)
 |-- lesao: string (nullable = true)
 |-- local_lesao: string (nullable = true)
 |-- obito: string (nullable = true)
 |-- cid_cod: string (nullable = true)
 |-- cid_desc: string (nullable = true)
 |-- cbo_cod: string (nullable = true)
 |-- cbo_desc: string (nullable = true)
 |-- cnae_cod: float (nullable = true)
 |-- cnae_desc: string (nullable = true)
 |-- cnpj_cei:

In [ ]:
df4_spark.show()

+-----------+-----------+---------------+--------------+---------+----------+-----------------+-----------+------+--------------------+------------------+---------+-------------------+-------------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------+--------------------+--------+--------------------+------------------+
|dt_acidente|dt_despacho|mes_afastamento|dt_emissao_cat|     sexo|   dt_nasc|    uf_empregador|uf_acidente|cod_uf|           municipio|          filiacao|beneficio|                cat|tipo_acidente|        agt_causador|               lesao|         local_lesao|obito|cid_cod|            cid_desc|cbo_cod|            cbo_desc|cnae_cod|           cnae_desc|          cnpj_cei|
+-----------+-----------+---------------+--------------+---------+----------+-----------------+-----------+------+--------------------+------------------+---------+-------------------+-------------+--------------------+--------------------+----------

In [ ]:
#selecionando algumas colunas para visualização
df4_spark.select(F.col('dt_despacho'), F.col('Sexo'), F.col('cnpj_cei')).show()

+-----------+---------+------------------+
|dt_despacho|     Sexo|          cnpj_cei|
+-----------+---------+------------------+
|        NaN| Feminino|57.571.275.002.570|
|        NaN|Masculino|19.578.086.000.162|
|        NaN|Masculino|               NaN|
|        NaN|Masculino|               NaN|
|        NaN|Masculino|               NaN|
|        NaN|Masculino|24.574.403.000.102|
|        NaN| Feminino|57.571.275.002.570|
|        NaN| Feminino|76.417.005.000.429|
|        NaN|Masculino|               NaN|
|        NaN| Feminino|               NaN|
|        NaN|Masculino|               NaN|
|        NaN|Masculino|               NaN|
|        NaN|Masculino|               NaN|
|        NaN| Feminino|57.571.275.002.570|
|        NaN|Masculino|               NaN|
|        NaN|Masculino|               NaN|
|        NaN| Feminino|               NaN|
|        NaN|Masculino|               NaN|
|        NaN|Masculino|               NaN|
|        NaN|Masculino|               NaN|
+----------

In [ ]:
#a coluna dt_despacho parece apresentar somente valores NaN, verificamos. Tinha outros valores além de NaN, então tá tudo certo.
df4_spark.select('dt_despacho').distinct().show(truncate=False)

+-----------+
|dt_despacho|
+-----------+
|2022/03    |
|2022/07    |
|2022/01    |
|2022/02    |
|NaN        |
|2022/06    |
|2022/04    |
|2022/10    |
|2022/11    |
|2022/05    |
+-----------+



In [ ]:
#A coluna cnpj_cei e dt_despacho apresentam muitos valores em NaN mas optamos por não dar drop.

In [ ]:
#Renomeamos duas colunas
df4_spark = ( df4_spark.withColumnRenamed('filiacao', 'contrato_trabalho')
                .withColumnRenamed('cnpj_cei', 'cnpj')
)

In [ ]:
df4_spark.show()

+-----------+-----------+---------------+--------------+---------+----------+-----------------+-----------+------+--------------------+------------------+---------+-------------------+-------------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------+--------------------+--------+--------------------+------------------+
|dt_acidente|dt_despacho|mes_afastamento|dt_emissao_cat|     sexo|   dt_nasc|    uf_empregador|uf_acidente|cod_uf|           municipio| contrato_trabalho|beneficio|                cat|tipo_acidente|        agt_causador|               lesao|         local_lesao|obito|cid_cod|            cid_desc|cbo_cod|            cbo_desc|cnae_cod|           cnae_desc|              cnpj|
+-----------+-----------+---------------+--------------+---------+----------+-----------------+-----------+------+--------------------+------------------+---------+-------------------+-------------+--------------------+--------------------+----------

In [ ]:
#Removendo linhas duplicadas no df4_spark
df4_spark.dropDuplicates()
total_linhas = df4_spark.count()
total_linhas_distintas = df4_spark.distinct().count()

print(f'O total de linhas excluídas foi de: {total_linhas - total_linhas_distintas}')

O total de linhas excluídas foi de: 89567


In [ ]:
#Salvando em formato csv
df4_spark.write.format('csv').save('df4_spark_tratado')

##Window Functions

In [ ]:
#Criando um ranking
drank = Window.partitionBy(F.col('uf_acidente')).orderBy(F.col('sexo').desc())
df4_spark_ran = df4_spark.withColumn('rank', F.dense_rank().over(drank)).show(10)


#Mostrando porcentagem
porcentagem = Window.partitionBy(F.col('tipo_acidente')).orderBy(F.col('agt_causador').desc())
df4_spark_por = df4_spark.withColumn('%', F.percent_rank().over(porcentagem)).show(10)

+-----------+-----------+---------------+--------------+----+----------+-------------+-----------+------+---------+-----------------+---------+----------+-------------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------+--------------------+--------+--------------------+----+----+
|dt_acidente|dt_despacho|mes_afastamento|dt_emissao_cat|sexo|   dt_nasc|uf_empregador|uf_acidente|cod_uf|municipio|contrato_trabalho|beneficio|       cat|tipo_acidente|        agt_causador|               lesao|         local_lesao|obito|cid_cod|            cid_desc|cbo_cod|            cbo_desc|cnae_cod|           cnae_desc|cnpj|rank|
+-----------+-----------+---------------+--------------+----+----------+-------------+-----------+------+---------+-----------------+---------+----------+-------------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------+--------------------+--------+--------------------+---

##Funções, groupby e filtros

In [ ]:
#Busca pelo agente causador com a palavra 'Motocicleta'
df5_spark = df4_spark.filter(F.col('agt_causador').contains('Motocicleta')).show(truncate=False)

#Verifica quais as lesões que aparecem no dataframe
df6_spark = df4_spark.select('lesao').distinct().show(truncate=False)

#Retornar Somente a Data do acidente, sexo, municipio, lesao e local_lesao
df7_spark = df4_spark.select(F.col('dt_acidente'), F.col('sexo'), F.col('municipio'), F.col('lesao'), F.col('local_lesao'), F.col('agt_causador')).show(truncate=False)

#Retornar a quantidade de pessoas da coluna sexo
df8_spark = df4_spark.groupBy('sexo').count().show()

#Ordenar a coluna data nascimento como por ordem ascendente
df9_spark = df4_spark.sort(F.col('dt_nasc').asc()).show(truncate=False)

+-----------+-----------+---------------+--------------+---------+----------+-----------------+----------------+------+-----------------------+-----------------+---------+-------------------+-------------+--------------------+------------------------------------------+---------------------------------------------+-----+-------+---------------------------------------+-------+--------------------------------------+--------+---------------------------------------------+------------------+
|dt_acidente|dt_despacho|mes_afastamento|dt_emissao_cat|sexo     |dt_nasc   |uf_empregador    |uf_acidente     |cod_uf|municipio              |contrato_trabalho|beneficio|cat                |tipo_acidente|agt_causador        |lesao                                     |local_lesao                                  |obito|cid_cod|cid_desc                               |cbo_cod|cbo_desc                              |cnae_cod|cnae_desc                                    |cnpj              |
+-----------+-----

##Insights

In [ ]:
#1 - verificamos o agt_causador correlacionando com o sexo do acidentado
df10_spark = df4_spark.groupBy('agt_causador', 'sexo').count().show(truncate=False)

#2 - verificamos a quantidade de acidentes por região
df11_spark = df4_spark.groupBy('uf_acidente').count().show(truncate=False)

#3 - vericamos a faixa etaria para ver qual a média de idade que mais sofreu acidentes 
df12_spark = df4_spark.groupBy('dt_nasc').count().show(truncate=False)

+---------------------------------------------+---------+-----+
|agt_causador                                 |sexo     |count|
+---------------------------------------------+---------+-----+
|Luminaria Globo Iampada   Mobiliario e Aces  |Feminino |7    |
|Substancia Quimica Material Produto Nic      |Masculino|277  |
|Ponte Viaduto   Edificio ou Estrutura        |Masculino|43   |
|Escada Permanente Cujos Degraus Permitem Apoi|NaN      |26   |
|Britador Moinho   Maquina                    |Feminino |6    |
|Escada Permanente Cujos Degraus Permitem Apoi|Feminino |2055 |
|Impacto Sofrido por Pes. de Objeto Projetad  |Masculino|1883 |
|Reacao do Corpo a Movimento Voluntario       |Feminino |510  |
|Motor Bomba Turbina Nic                      |Masculino|132  |
|Tunel                                        |Feminino |6    |
|Equip. Emissor de Radiacao não Ionizante     |Feminino |1    |
|Tesoura Guilhotina Maquina de Cortar   Maqu  |Masculino|283  |
|Texteis   Inclui Fibras Animais Apos O 

#🚀 MySQL

In [ ]:
#Instalando o mysql
!pip install mysql-connector-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.5 MB 1.3 MB/s 


In [ ]:
import mysql.connector

In [ ]:
#Criando os novos df's m SQL

df_sql = df.to_sql
df2_sql = df2.to_sql
df3_sql = df3.to_sql

In [ ]:
spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [ ]:
spark.sql('create database projeto_aquecimento').show()

++
||
++
++



In [ ]:
spark.sql('use projeto_aquecimento')

DataFrame[]

In [ ]:
df4_spark.write.saveAsTable('projeto_aquecimento')

In [ ]:
spark.sql('select * from projeto_aquecimento').show(2)

+-----------+-----------+---------------+--------------+---------+----------+-------------+-----------+------+--------------+-----------------+---------+-------------------+-------------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------+--------------------+--------+--------------------+------------------+
|dt_acidente|dt_despacho|mes_afastamento|dt_emissao_cat|     sexo|   dt_nasc|uf_empregador|uf_acidente|cod_uf|     municipio|contrato_trabalho|beneficio|                cat|tipo_acidente|        agt_causador|               lesao|         local_lesao|obito|cid_cod|            cid_desc|cbo_cod|            cbo_desc|cnae_cod|           cnae_desc|              cnpj|
+-----------+-----------+---------------+--------------+---------+----------+-------------+-----------+------+--------------+-----------------+---------+-------------------+-------------+--------------------+--------------------+--------------------+-----+-------+--------